# **Deep Learning**

# Lab assignment: the hunger games

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

In this assignment we will face a challenging image classification problem, building a deep learning model that is able to classify different kinds of foods. Let the hunger games begin!

## Guidelines

The following code will embed any plots into the notebook instead of generating a new window:

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

## Data acquisition

We will use a food images dataset available at [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). To download it you will need to create a user account in Kaggle, and obtain your API credential by following the instructions on [this section](https://github.com/Kaggle/kaggle-api#api-credentials). Once you have your credentials JSON file, you can inform this notebook of them by setting the appropriate enviroment variables, as follows

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Once this is done, you will be able to download the dataset to this computer using the command

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
where you should write a valid directory in your computer in "YOUR_LOCAL_FOLDER". If you are fine with downloading the data in the same folder as this notebook, just skip the `-p YOUR_LOCAL_FOLDER` part of the command.

In [2]:
import os

os.environ["KAGGLE_USERNAME"] = "XXX"
os.environ["KAGGLE_KEY"] = "XXX"

!kaggle datasets download trolukovich/food11-image-dataset --unzip 

 99% 1.07G/1.08G [00:07<00:00, 155MB/s]
100% 1.08G/1.08G [00:07<00:00, 152MB/s]


In [3]:
TRAINDIR = "/content/training"
VALDIR = "/content/validation"
TESTDIR = "/content/evaluation"

Take now a look into the folder where you downloaded the data. You will find it is made up of three subfolders:

* **training**, containing the images to use to train the model.
* **validation**, containing additional images you could use as more training data, or for some kind of validation strategy such as Early Stopping.
* **evaluation**, containing the images you must use to test your model. Images in this folder can **only** be used to measure the model performance after the training procedure is completed.

Furthermore, within each one of these folders you will find one folder for each one of the 11 classes in this problem:

* Bread
* Dairy product
* Dessert
* Egg
* Fried food
* Meat
* Noodles-Pasta
* Rice
* Seafood
* Soup
* Vegetable-Fruit

This is a standard structure for organizing image datasets: one folder per class. To easen the following data processing steps, let us define some variables telling us where the data is located.

Let's plot a random sample of training images from each class, using the ipyplot package. If you are running this notebook in Google Colab, you will need to install this package first with

    !pip install ipyplot

You can inspect each class by clicking the different tabs in the interface that will appear when running the following cell.

In [ ]:
!pip install ipyplot

In [ ]:
from glob import glob
import ipyplot
import numpy as np

all_images = glob(f"{TRAINDIR}/*/*.jpg")  # Get all image paths
np.random.shuffle(all_images)  # Randomize to show different images each run
all_labels = [f.split("/")[-2] for f in all_images]  # Extract class names from path

ipyplot.plot_class_tabs(all_images, all_labels, max_imgs_per_tab=6, img_width=300, force_b64=True)

### Class reduction

To make the problem more approachable for this exercise, we will focus on just six classes: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` and `Meat`. To do so, we will delete from the downloaded data the folders from other classes.

In [6]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

Deleting /content/evaluation/Soup...
Deleting /content/evaluation/Seafood...
Deleting /content/evaluation/Rice...
Deleting /content/evaluation/Vegetable-Fruit...
Deleting /content/evaluation/Noodles-Pasta...
Deleting /content/validation/Soup...
Deleting /content/validation/Seafood...
Deleting /content/validation/Rice...
Deleting /content/validation/Vegetable-Fruit...
Deleting /content/validation/Noodles-Pasta...
Deleting /content/training/Soup...
Deleting /content/training/Seafood...
Deleting /content/training/Rice...
Deleting /content/training/Vegetable-Fruit...
Deleting /content/training/Noodles-Pasta...


## Image processing from files

This dataset of images is large, with images of larger resolution than in the tutorial MNIST problem, each one having different sizes and image ratios. Also, while for MNIST we had a keras function that prepared the dataset for us, this time we will need to do some loading and image processing work.

A convenient way to do this work is through the use of Keras `image_dataset_from_directory` function. This function creates a TensorFlow `Dataset` with the images in the directory, loading images dynamically only when the neural network needs to use them, and also allowing us to specify some useful preprocessing options.

For example, we can create such a `Dataset` with the data in the training folder:

In [46]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Found 6082 files belonging to 6 classes.


Note the parameters used to configure the dataset:

* The **directory** from which to load the images.
* An **image size** that will be used to resize all the images to a common resolution, here 32x32.
* The **size of the batches** of images to be generated. Note we define this parameter here instead in the network fit step, as the `Dataset` will make use of this information to keep in memory only a few batches of images at the same time in order to save memory.
* The **label mode**, that is, the encoding used for the labels. `categorical` means we will use one-hot encoding.

A `Dataset` object works like a Python generator, which means we can iterate over it to obtain batches of processed images. For instance, to get the first batch

In [47]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Shape of input batch: (64, 32, 32, 3)
Shape of output batch: (64, 6)
Input batch:
[[[[108.03125  10.90625   5.6875 ]
   [215.1875  163.875   159.625  ]
   [  5.5625    2.1875    8.65625]
   ...
   [217.53125 228.53125 230.53125]
   [222.      233.      235.     ]
   [224.09375 235.09375 237.09375]]

  [[  0.        1.5       0.     ]
   [  0.        0.28125   0.     ]
   [  0.        0.        0.     ]
   ...
   [217.96875 228.46875 231.46875]
   [220.      234.      237.     ]
   [222.      236.      239.     ]]

  [[  7.9375    8.4375   10.4375 ]
   [  0.        0.28125   0.     ]
   [  0.        0.        0.     ]
   ...
   [215.      225.      224.     ]
   [216.      226.      227.     ]
   [217.5     226.5     231.5    ]]

  ...

  [[189.21875 205.21875 205.21875]
   [188.34375 203.46875 203.90625]
   [192.03125 206.03125 207.03125]
   ...
   [212.78125 226.78125 235.78125]
   [221.4375  235.4375  246.4375 ]
   [214.375   228.375   239.375  ]]

  [[193.59375 207.59375 208.59375]


We can see that indeed the generator produces a tensor of the appropriate dimensions with the inputs for the neural network, and that the outputs have also been properly codified in one-hot form. However, there is still an issue with the data: the pixel values are in the range [0, 255], which might produce training problems. We will solve this later in the network definition. For now, let's move on and prepare a funcion that builds the training, validation and test datasets.

In [7]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

def  create_datasets(traindir, valdir, testdir, image_size, batch_size):

  train_dataset = image_dataset_from_directory(
      traindir, 
      image_size = (image_size, image_size),
      batch_size = batch_size, 
      label_mode = 'categorical'
  )

  val_dataset = image_dataset_from_directory(
      valdir, 
      image_size = (image_size, image_size),
      batch_size = batch_size, 
      label_mode = 'categorical'
  )
    
  test_dataset = image_dataset_from_directory(
      testdir, 
      image_size = (image_size, image_size),
      batch_size = batch_size, 
      label_mode = 'categorical'
  )

  return train_dataset, val_dataset, test_dataset

Let's test if the function you just implemented works correctly

In [ ]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


Now that we have our datasets we can train a deep learning model using them! For illustration purposes, let's build an extremely simple convolutional network. Note how we have added a special pre-processing layer `Rescaling` that takes care of normalizing the data to the range [0, 1].

Be careful! This network will work, but has some flaws in its design you might want to fix in the network you will desing later in this notebook.

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

The `fit` method of a Keras model can receive a `Dataset` with training data, instead of a pair of tensors with (inputs, outputs). Since when building the `Dataset` we already specified the batch size, we don't need to do it now.

In [ ]:
model.fit(train_dataset, epochs=1)

96/96 [==============================] - 10s 89ms/step - loss: 1.7142 - accuracy: 0.2669


Similarly, we can evaluate the performance of our model over our test `Dataset` as follows

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 9s 219ms/step - loss: 1.9268 - accuracy: 0.2343
Loss 1.93, accuracy 23.4%


The accuracy might seem poor, but take into account we have used a very simple model and this problem has 6 classes. Will you be able to do better?

## Building your network

#### **"LeNet_01" (RED NEURONAL CONVOLUCIONAL)**



In [48]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_01"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling

image_size = 32

img_rows = 32
img_cols = 32
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_01 = Sequential()

LeNet_01.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_01.add(Convolution2D(
    32, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_01.add(BatchNormalization())

LeNet_01.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_01.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_01.add(BatchNormalization())

LeNet_01.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [49]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_01.add(Flatten())
LeNet_01.add(Dense(256, activation="relu"))
LeNet_01.add(Dropout(0.75))
LeNet_01.add(Dense(6, activation="softmax"))

In [50]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_01.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
LeNet_01.fit(
    train_dataset, # Training data
    batch_size=128, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
96/96 - 7s - loss: 2.0403 - accuracy: 0.2831
Epoch 2/20
96/96 - 6s - loss: 1.6204 - accuracy: 0.3206
Epoch 3/20
96/96 - 6s - loss: 1.5684 - accuracy: 0.3486
Epoch 4/20
96/96 - 6s - loss: 1.5299 - accuracy: 0.3528
Epoch 5/20
96/96 - 6s - loss: 1.5056 - accuracy: 0.3742
Epoch 6/20
96/96 - 6s - loss: 1.4590 - accuracy: 0.3869
Epoch 7/20
96/96 - 6s - loss: 1.4355 - accuracy: 0.4140
Epoch 8/20
96/96 - 6s - loss: 1.4247 - accuracy: 0.4107
Epoch 9/20
96/96 - 6s - loss: 1.4029 - accuracy: 0.4226
Epoch 10/20
96/96 - 6s - loss: 1.3844 - accuracy: 0.4193
Epoch 11/20
96/96 - 6s - loss: 1.3358 - accuracy: 0.4617
Epoch 12/20
96/96 - 6s - loss: 1.3118 - accuracy: 0.4625
Epoch 13/20
96/96 - 6s - loss: 1.2812 - accuracy: 0.4832
Epoch 14/20
96/96 - 6s - loss: 1.2342 - accuracy: 0.5046
Epoch 15/20
96/96 - 6s - loss: 1.2008 - accuracy: 0.5178
Epoch 16/20
96/96 - 6s - loss: 1.1884 - accuracy: 0.5286
Epoch 17/20
96/96 - 6s - loss: 1.2102 - accuracy: 0.5148
Epoch 18/20
96/96 - 6s - loss: 1.1744 - 

In [ ]:
probs = LeNet_01.predict(test_dataset)

In [ ]:
probs

array([[3.20054293e-02, 6.02992804e-05, 6.95663393e-02, 1.59418141e-03,
        9.47257411e-03, 8.87301266e-01],
       [1.58360433e-02, 1.45257695e-03, 9.06817615e-01, 4.99280286e-04,
        1.48611376e-04, 7.52458572e-02],
       [6.65064726e-04, 8.97512131e-04, 9.98250544e-01, 8.37463303e-05,
        6.31299145e-07, 1.02385959e-04],
       ...,
       [1.12698786e-01, 3.08150407e-02, 3.78167838e-01, 6.13643602e-02,
        7.21549913e-02, 3.44798952e-01],
       [4.86004800e-01, 1.24727711e-02, 2.14583009e-01, 2.18560278e-01,
        6.49203211e-02, 3.45888757e-03],
       [2.86728472e-01, 1.26311732e-02, 2.43173897e-01, 1.07253335e-01,
        2.45201886e-01, 1.05011284e-01]], dtype=float32)

In [ ]:
score = LeNet_01.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

33/33 [==============================] - 3s 59ms/step - loss: 1.3601 - accuracy: 0.5300
Test loss 1.36005437374115
Test accuracy 0.5299516916275024


### **Conclusiones:**

Dado que el modelo **"LetNet_01"** alcanza el rendimiento buscado se toma como válido.

### **PRO**



Ya tenemos un "accuracy" en train en torno al 40%, siendo el de test en torno al 34% por lo que tenemos algo de "overfitting" en la red neuronal entrenada. Para aumentar el "accuracy" y tratar de reducir el "overfitting" de la red vamos a definir una nueva, tratando de aplicar en ella algunos parámetros adicionales.

Uno de los aspectos que vamos a utilizar es aumentar el tamaño de las imágenes de entrada para tratar de conseguir más datos de cada imagen. Para tratar esas imágenes más grandes vamos a incrementar el nº de capas de procesamiento de nuestra red y vamos a incluir otras técnicas para aumentar la variabilidad del dataset de entrenamiento.


In [ ]:
# Datasets imágenes tamaño 128x128

import tensorflow as tf

train_dataset128, val_dataset128, test_dataset128 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset128, tf.data.Dataset)
assert isinstance(val_dataset128, tf.data.Dataset)
assert isinstance(test_dataset128, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [ ]:
# Datasets imágenes tamaño 256x256

import tensorflow as tf

train_dataset256, val_dataset256, test_dataset256 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=256, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset256, tf.data.Dataset)
assert isinstance(val_dataset256, tf.data.Dataset)
assert isinstance(test_dataset256, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


#### **"LeNet_02" (RED NEURONAL CONVOLUCIONAL)**



In [157]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_02"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

image_size=128

img_rows = 128
img_cols = 128
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_02 = Sequential()

LeNet_02.add(RandomFlip(mode="horizontal"))
LeNet_02.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_02.add(RandomRotation(factor=(-0.1,0.1)))
LeNet_02.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#LeNet_02.add(RandomCrop(height=85, width=85))

LeNet_02.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_02.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_02.add(BatchNormalization())

LeNet_02.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_02.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_02.add(BatchNormalization())

LeNet_02.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_02.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_02.add(BatchNormalization())

LeNet_02.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_02.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_02.add(BatchNormalization())

LeNet_02.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [158]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_02.add(Flatten())
LeNet_02.add(Dense(256, activation="relu"))
LeNet_02.add(Dropout(0.45))
LeNet_02.add(Dense(6, activation="softmax"))

In [159]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_02.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [160]:
from keras.callbacks import EarlyStopping

LeNet_02.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=256, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 11s - loss: 1.9719 - accuracy: 0.3053 - val_loss: 2.4724 - val_accuracy: 0.2367
Epoch 2/20
191/191 - 8s - loss: 1.6162 - accuracy: 0.3366 - val_loss: 1.6430 - val_accuracy: 0.3083
Epoch 3/20
191/191 - 8s - loss: 1.5775 - accuracy: 0.3520 - val_loss: 1.4322 - val_accuracy: 0.4303
Epoch 4/20
191/191 - 8s - loss: 1.5411 - accuracy: 0.3699 - val_loss: 1.4646 - val_accuracy: 0.3971
Epoch 5/20
191/191 - 8s - loss: 1.5202 - accuracy: 0.3731 - val_loss: 1.4421 - val_accuracy: 0.4108
Epoch 6/20
191/191 - 8s - loss: 1.4761 - accuracy: 0.3976 - val_loss: 2.9209 - val_accuracy: 0.3321
Epoch 7/20
191/191 - 8s - loss: 1.4677 - accuracy: 0.4127 - val_loss: 1.4540 - val_accuracy: 0.4004
Epoch 8/20
191/191 - 8s - loss: 1.4612 - accuracy: 0.4152 - val_loss: 1.4212 - val_accuracy: 0.4265
Epoch 9/20
191/191 - 8s - loss: 1.4331 - accuracy: 0.4217 - val_loss: 1.6428 - val_accuracy: 0.3676
Epoch 10/20
191/191 - 8s - loss: 1.4119 - accuracy: 0.4479 - val_loss: 1.4675 - val_accuracy: 0.434

In [161]:
probs = LeNet_02.predict(test_dataset128)

In [162]:
probs

array([[1.8054570e-01, 7.8031518e-02, 3.6543319e-01, 9.3800917e-02,
        1.4711839e-01, 1.3507034e-01],
       [2.0207924e-01, 1.5819007e-01, 2.2820020e-01, 1.3225992e-01,
        2.1580835e-01, 6.3462205e-02],
       [9.3618065e-02, 3.0819002e-01, 2.6223880e-01, 2.3721616e-01,
        6.5871388e-02, 3.2865580e-02],
       ...,
       [1.4244377e-04, 3.7992286e-04, 5.9555155e-01, 1.5435053e-05,
        4.7771879e-05, 4.0386292e-01],
       [1.5800887e-01, 6.8806656e-02, 2.8750533e-01, 6.7117617e-02,
        3.0358168e-01, 1.1497986e-01],
       [9.7606041e-02, 6.6495100e-05, 5.3270608e-02, 7.2183408e-04,
        3.8696048e-01, 4.6137455e-01]], dtype=float32)

In [163]:
score = LeNet_02.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 2s 29ms/step - loss: 1.2246 - accuracy: 0.5440
Test loss 1.224624752998352
Test accuracy 0.5439613461494446


#### **"LeNet_03" (RED NEURONAL CONVOLUCIONAL)**



In [164]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_03"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

image_size=256

img_rows = 256
img_cols = 256
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_03 = Sequential()

LeNet_03.add(RandomFlip(mode="horizontal"))
LeNet_03.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_03.add(RandomRotation(factor=(-0.1,0.1)))
LeNet_03.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_03.add(RandomCrop(height=128, width=128))

LeNet_03.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_03.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_03.add(BatchNormalization())

LeNet_03.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_03.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_03.add(BatchNormalization())

LeNet_03.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_03.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_03.add(BatchNormalization())

LeNet_03.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_03.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_03.add(BatchNormalization())

LeNet_03.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_03.add(Convolution2D(
    256, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_03.add(BatchNormalization())

LeNet_03.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [165]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_03.add(Flatten())
LeNet_03.add(Dense(512, activation="relu"))
LeNet_03.add(Dropout(0.4))
LeNet_03.add(Dense(6, activation="softmax"))

In [166]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_03.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [167]:
from keras.callbacks import EarlyStopping

LeNet_03.fit(
    train_dataset256, # Training data
    validation_data=val_dataset256,
    batch_size=256, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 14s - loss: 2.1306 - accuracy: 0.2728 - val_loss: 2.3654 - val_accuracy: 0.2415
Epoch 2/20
191/191 - 12s - loss: 1.6832 - accuracy: 0.3170 - val_loss: 1.6588 - val_accuracy: 0.2946
Epoch 3/20
191/191 - 12s - loss: 1.6336 - accuracy: 0.3351 - val_loss: 1.5410 - val_accuracy: 0.3610
Epoch 4/20
191/191 - 12s - loss: 1.5992 - accuracy: 0.3542 - val_loss: 1.5077 - val_accuracy: 0.3980
Epoch 5/20
191/191 - 12s - loss: 1.5610 - accuracy: 0.3703 - val_loss: 1.4402 - val_accuracy: 0.4336
Epoch 6/20
191/191 - 12s - loss: 1.5649 - accuracy: 0.3775 - val_loss: 1.6027 - val_accuracy: 0.3776
Epoch 7/20
191/191 - 12s - loss: 1.5391 - accuracy: 0.3800 - val_loss: 1.4879 - val_accuracy: 0.4293
Epoch 8/20
191/191 - 12s - loss: 1.5233 - accuracy: 0.4025 - val_loss: 1.8380 - val_accuracy: 0.3411
Epoch 9/20
191/191 - 12s - loss: 1.4979 - accuracy: 0.4043 - val_loss: 1.5867 - val_accuracy: 0.4260
Epoch 10/20
191/191 - 12s - loss: 1.5081 - accuracy: 0.4046 - val_loss: 1.7172 - val_accura

In [168]:
probs = LeNet_03.predict(test_dataset256)

In [169]:
probs

array([[0.22204264, 0.10496853, 0.21231613, 0.3238781 , 0.10044695,
        0.03634764],
       [0.1755103 , 0.09802694, 0.15507923, 0.42139372, 0.11896059,
        0.03102923],
       [0.14411916, 0.04601154, 0.15305254, 0.16551422, 0.4060129 ,
        0.08528964],
       ...,
       [0.10149255, 0.05892523, 0.5131337 , 0.11614926, 0.03620072,
        0.1740985 ],
       [0.22552486, 0.02892091, 0.3054738 , 0.06075696, 0.05740061,
        0.3219228 ],
       [0.1307102 , 0.1247412 , 0.32563496, 0.24119198, 0.07085649,
        0.1068651 ]], dtype=float32)

In [170]:
score = LeNet_03.evaluate(test_dataset256)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 45ms/step - loss: 1.4166 - accuracy: 0.4556
Test loss 1.4165751934051514
Test accuracy 0.4555555582046509


#### **"LeNet_04" (RED NEURONAL CONVOLUCIONAL)**



In [207]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_04"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

image_size=256

img_rows = 256
img_cols = 256
kernel_size = 4 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_04 = Sequential()

LeNet_04.add(RandomFlip(mode="horizontal"))
LeNet_04.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_04.add(RandomRotation(factor=(-0.1,0.1)))
LeNet_04.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_04.add(RandomCrop(height=128, width=128))

LeNet_04.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_04.add(Convolution2D(
    120, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_04.add(BatchNormalization())

LeNet_04.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_04.add(Convolution2D(
    256, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_04.add(BatchNormalization())

LeNet_04.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_04.add(Convolution2D(
    360, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_04.add(BatchNormalization())

LeNet_04.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_04.add(Convolution2D(
    360, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_04.add(BatchNormalization())

LeNet_04.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_04.add(Convolution2D(
    360, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_04.add(BatchNormalization())

LeNet_04.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [208]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_04.add(Flatten())
LeNet_04.add(Dense(512, activation="relu"))
LeNet_04.add(Dropout(0.4))
LeNet_04.add(Dense(256, activation="relu"))
LeNet_04.add(Dropout(0.4))
LeNet_04.add(Dense(6, activation="softmax"))

In [209]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_04.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [210]:
from keras.callbacks import EarlyStopping

LeNet_04.fit(
    train_dataset256, # Training data
    validation_data=val_dataset256,
    batch_size=64, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 20s - loss: 1.8461 - accuracy: 0.2673 - val_loss: 1.7880 - val_accuracy: 0.1959
Epoch 2/20
191/191 - 17s - loss: 1.6903 - accuracy: 0.2895 - val_loss: 1.6405 - val_accuracy: 0.3083
Epoch 3/20
191/191 - 17s - loss: 1.6574 - accuracy: 0.3017 - val_loss: 1.6577 - val_accuracy: 0.2766
Epoch 4/20
191/191 - 17s - loss: 1.6573 - accuracy: 0.2948 - val_loss: 1.5587 - val_accuracy: 0.3710
Epoch 5/20
191/191 - 17s - loss: 1.6285 - accuracy: 0.3172 - val_loss: 1.5269 - val_accuracy: 0.3743
Epoch 6/20
191/191 - 17s - loss: 1.6214 - accuracy: 0.3195 - val_loss: 1.7345 - val_accuracy: 0.3041
Epoch 7/20
191/191 - 17s - loss: 1.6137 - accuracy: 0.3402 - val_loss: 1.6176 - val_accuracy: 0.3515
Epoch 8/20
191/191 - 17s - loss: 1.6051 - accuracy: 0.3320 - val_loss: 1.5033 - val_accuracy: 0.4198
Epoch 9/20
191/191 - 17s - loss: 1.6003 - accuracy: 0.3446 - val_loss: 1.7035 - val_accuracy: 0.2789
Epoch 10/20
191/191 - 17s - loss: 1.6001 - accuracy: 0.3463 - val_loss: 1.4779 - val_accura

In [211]:
probs = LeNet_04.predict(test_dataset256)

In [212]:
probs

array([[0.07110782, 0.01232759, 0.45568112, 0.02015596, 0.02725954,
        0.41346794],
       [0.21364509, 0.05447133, 0.14137186, 0.3384344 , 0.12619202,
        0.12588526],
       [0.14029978, 0.05671131, 0.4947207 , 0.09293626, 0.06833907,
        0.14699288],
       ...,
       [0.23054138, 0.01920655, 0.22091192, 0.08061139, 0.1279186 ,
        0.3208101 ],
       [0.1851802 , 0.12959221, 0.29531616, 0.18701449, 0.07794872,
        0.12494823],
       [0.22437544, 0.0048871 , 0.09598884, 0.04297594, 0.32951248,
        0.30226022]], dtype=float32)

In [213]:
score = LeNet_04.evaluate(test_dataset256)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 49ms/step - loss: 1.5260 - accuracy: 0.3957
Test loss 1.5260251760482788
Test accuracy 0.395652174949646


#### **"LeNet_05" (RED NEURONAL CONVOLUCIONAL)**



In [178]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_05"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

image_size=128

img_rows = 128
img_cols = 128
kernel_size = 4 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_05 = Sequential()

LeNet_05.add(RandomFlip(mode="horizontal"))
LeNet_05.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_05.add(RandomRotation(factor=(-0.1,0.1)))
LeNet_05.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_05.add(RandomCrop(height=128, width=128))

LeNet_05.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_05.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_05.add(BatchNormalization())

LeNet_05.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_05.add(Convolution2D(
    256, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_05.add(BatchNormalization())

LeNet_05.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_05.add(Convolution2D(
    360, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_05.add(BatchNormalization())

LeNet_05.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_05.add(Convolution2D(
    360, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_05.add(BatchNormalization())

LeNet_05.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [179]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_05.add(Flatten())
LeNet_05.add(Dense(4096, activation="relu"))
LeNet_05.add(Dropout(0.4))
LeNet_05.add(Dense(4096, activation="relu"))
LeNet_05.add(Dropout(0.4))
LeNet_05.add(Dense(6, activation="softmax"))

In [180]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_05.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [181]:
from keras.callbacks import EarlyStopping

LeNet_05.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=64, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 19s - loss: 3.4706 - accuracy: 0.2803 - val_loss: 2.2343 - val_accuracy: 0.1902
Epoch 2/20
191/191 - 16s - loss: 1.7233 - accuracy: 0.3131 - val_loss: 1.8644 - val_accuracy: 0.2310
Epoch 3/20
191/191 - 16s - loss: 1.7584 - accuracy: 0.3093 - val_loss: 1.6077 - val_accuracy: 0.3316
Epoch 4/20
191/191 - 16s - loss: 1.7404 - accuracy: 0.3114 - val_loss: 1.8338 - val_accuracy: 0.2514
Epoch 5/20
191/191 - 16s - loss: 1.7306 - accuracy: 0.3213 - val_loss: 2.2330 - val_accuracy: 0.2481
Epoch 6/20
191/191 - 16s - loss: 1.7259 - accuracy: 0.3119 - val_loss: 1.7394 - val_accuracy: 0.2448
Epoch 7/20
191/191 - 16s - loss: 1.6997 - accuracy: 0.3265 - val_loss: 2.0706 - val_accuracy: 0.2552
Epoch 8/20
191/191 - 16s - loss: 1.6930 - accuracy: 0.3269 - val_loss: 1.5868 - val_accuracy: 0.3657
Epoch 9/20
191/191 - 16s - loss: 1.6590 - accuracy: 0.3415 - val_loss: 1.5248 - val_accuracy: 0.3871
Epoch 10/20
191/191 - 16s - loss: 1.6389 - accuracy: 0.3430 - val_loss: 1.5765 - val_accura

In [182]:
probs = LeNet_05.predict(test_dataset128)

In [183]:
probs

array([[0.3125909 , 0.02136292, 0.06988531, 0.16051504, 0.35256255,
        0.08308335],
       [0.09492423, 0.26242226, 0.38774174, 0.10518987, 0.06767455,
        0.0820474 ],
       [0.05093157, 0.25259507, 0.5651257 , 0.085617  , 0.01740827,
        0.02832237],
       ...,
       [0.11123084, 0.15662013, 0.34573632, 0.14774717, 0.08865571,
        0.15000978],
       [0.4323015 , 0.05099925, 0.04530528, 0.04340688, 0.4134024 ,
        0.01458475],
       [0.09545962, 0.22143811, 0.3610211 , 0.2195411 , 0.05419194,
        0.0483482 ]], dtype=float32)

In [184]:
score = LeNet_05.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 32ms/step - loss: 1.5507 - accuracy: 0.3836
Test loss 1.5506917238235474
Test accuracy 0.38357487320899963


#### **"LeNet_06" (RED NEURONAL CONVOLUCIONAL)**



In [185]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_06"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.models import Sequential

image_size=128

img_rows = 128
img_cols = 128
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_06 = Sequential()

LeNet_06.add(RandomFlip(mode="horizontal"))
LeNet_06.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_06.add(RandomRotation(factor=(-0.1,0.1)))
LeNet_06.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_06.add(RandomCrop(height=85, width=85))

LeNet_06.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_06.add(Convolution2D(
    32, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_06.add(BatchNormalization())

LeNet_06.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_06.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_06.add(BatchNormalization())

LeNet_06.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_06.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_06.add(BatchNormalization())

LeNet_06.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [186]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_06.add(Flatten())
LeNet_06.add(Dense(1024, activation="relu"))
LeNet_06.add(Dropout(0.5))
LeNet_06.add(Dense(6, activation="softmax"))

In [187]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_06.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [188]:
from keras.callbacks import EarlyStopping

LeNet_06.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=64, # Batch size for the optimizer algorithm
    epochs=50, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=10, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/50
191/191 - 11s - loss: 3.4718 - accuracy: 0.2631 - val_loss: 1.9086 - val_accuracy: 0.2410
Epoch 2/50
191/191 - 8s - loss: 1.7077 - accuracy: 0.3086 - val_loss: 1.7826 - val_accuracy: 0.2462
Epoch 3/50
191/191 - 8s - loss: 1.6385 - accuracy: 0.3200 - val_loss: 1.5416 - val_accuracy: 0.3643
Epoch 4/50
191/191 - 8s - loss: 1.6143 - accuracy: 0.3269 - val_loss: 1.5050 - val_accuracy: 0.4075
Epoch 5/50
191/191 - 8s - loss: 1.6178 - accuracy: 0.3420 - val_loss: 1.5577 - val_accuracy: 0.3491
Epoch 6/50
191/191 - 8s - loss: 1.6017 - accuracy: 0.3441 - val_loss: 1.5801 - val_accuracy: 0.3752
Epoch 7/50
191/191 - 8s - loss: 1.5858 - accuracy: 0.3463 - val_loss: 1.4826 - val_accuracy: 0.4089
Epoch 8/50
191/191 - 8s - loss: 1.5745 - accuracy: 0.3565 - val_loss: 1.5416 - val_accuracy: 0.3700
Epoch 9/50
191/191 - 8s - loss: 1.5767 - accuracy: 0.3602 - val_loss: 1.8172 - val_accuracy: 0.3202
Epoch 10/50
191/191 - 8s - loss: 1.5570 - accuracy: 0.3665 - val_loss: 1.5073 - val_accuracy: 0.377

In [189]:
probs = LeNet_06.predict(test_dataset128)

In [190]:
probs

array([[0.23589157, 0.0440163 , 0.16817558, 0.17633902, 0.17667933,
        0.19889818],
       [0.2489544 , 0.01607711, 0.16696072, 0.09495644, 0.32523087,
        0.14782053],
       [0.11172562, 0.11255375, 0.4448231 , 0.17382877, 0.09572788,
        0.06134091],
       ...,
       [0.40642926, 0.01189675, 0.05434922, 0.11631133, 0.3557462 ,
        0.05526725],
       [0.14000016, 0.05717981, 0.2731279 , 0.12943642, 0.1030966 ,
        0.29715914],
       [0.41237572, 0.01341014, 0.07585468, 0.20993933, 0.26378378,
        0.02463632]], dtype=float32)

In [191]:
score = LeNet_06.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 2s 29ms/step - loss: 1.3986 - accuracy: 0.4319
Test loss 1.3985600471496582
Test accuracy 0.4318840503692627


#### **"LeNet_07" (RED NEURONAL CONVOLUCIONAL)**



In [251]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "LeNet_07"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.models import Sequential

image_size=128

img_rows = 128
img_cols = 128
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

LeNet_07 = Sequential()

LeNet_07.add(RandomFlip(mode="horizontal"))
#LeNet_07.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
LeNet_07.add(RandomRotation(factor=(-0.2,0.2)))
#LeNet_07.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#LeNet_07.add(RandomCrop(height=85, width=85))
#LeNet_07.add(RandomContrast(factor=0.2))

#LeNet_07.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))

LeNet_07.add(Convolution2D(
    32, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_07.add(BatchNormalization())

LeNet_07.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_07.add(Convolution2D(
    64, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_07.add(BatchNormalization())

LeNet_07.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

LeNet_07.add(Convolution2D(
    128, # Number convolution channels to generate
    (kernel_size, kernel_size), # Size of convolution kernels
    padding='valid', # Strategy to deal with borders
    input_shape=(img_rows, img_cols, 3), # Size = image rows x image columns x channels
    activation="relu"  # Activation function after the convolution
))

LeNet_07.add(BatchNormalization())

LeNet_07.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

In [252]:
from keras.layers.core import Dense, Activation, Flatten, Dropout

LeNet_07.add(Flatten())
LeNet_07.add(Dense(1024, activation="relu"))
LeNet_07.add(Dropout(0.4))
LeNet_07.add(Dense(1024, activation="relu"))
LeNet_07.add(Dropout(0.4))
LeNet_07.add(Dense(6, activation="softmax"))

In [253]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
LeNet_07.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [254]:
from keras.callbacks import EarlyStopping

LeNet_07.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=64, # Batch size for the optimizer algorithm
    epochs=40, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/40
191/191 - 10s - loss: 4.4189 - accuracy: 0.2647 - val_loss: 1.6051 - val_accuracy: 0.3036
Epoch 2/40
191/191 - 8s - loss: 1.7472 - accuracy: 0.3178 - val_loss: 1.5938 - val_accuracy: 0.3311
Epoch 3/40
191/191 - 8s - loss: 1.6055 - accuracy: 0.3418 - val_loss: 1.5421 - val_accuracy: 0.3397
Epoch 4/40
191/191 - 8s - loss: 1.5498 - accuracy: 0.3760 - val_loss: 1.5321 - val_accuracy: 0.3705
Epoch 5/40
191/191 - 8s - loss: 1.5060 - accuracy: 0.3870 - val_loss: 1.4908 - val_accuracy: 0.3781
Epoch 6/40
191/191 - 8s - loss: 1.4875 - accuracy: 0.3935 - val_loss: 1.5005 - val_accuracy: 0.4004
Epoch 7/40
191/191 - 8s - loss: 1.4657 - accuracy: 0.4138 - val_loss: 1.6967 - val_accuracy: 0.3814
Epoch 8/40
191/191 - 8s - loss: 1.4613 - accuracy: 0.4074 - val_loss: 1.4430 - val_accuracy: 0.4421
Epoch 9/40
191/191 - 8s - loss: 1.4475 - accuracy: 0.4268 - val_loss: 1.5721 - val_accuracy: 0.3847
Epoch 10/40
191/191 - 8s - loss: 1.4084 - accuracy: 0.4433 - val_loss: 1.4712 - val_accuracy: 0.412

In [255]:
probs = LeNet_07.predict(test_dataset128)

In [256]:
probs

array([[4.68821526e-02, 3.15077491e-02, 2.28998080e-01, 1.17171695e-02,
        7.62218833e-02, 6.04672968e-01],
       [5.72833240e-01, 9.47223231e-03, 1.34899810e-01, 1.64995879e-01,
        5.98481409e-02, 5.79506755e-02],
       [7.24207098e-07, 1.41055324e-07, 5.08181984e-03, 3.26416716e-09,
        3.50917981e-06, 9.94913697e-01],
       ...,
       [1.46231383e-01, 2.31810600e-01, 2.25347057e-01, 2.00837970e-01,
        1.40768930e-01, 5.50040491e-02],
       [1.64767459e-01, 1.54148385e-01, 3.88039768e-01, 1.66465640e-01,
        1.16331734e-01, 1.02470387e-02],
       [3.73025192e-04, 8.78274077e-05, 4.87759802e-03, 1.22566911e-04,
        4.88753198e-04, 9.94050205e-01]], dtype=float32)

In [257]:
score = LeNet_07.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 2s 29ms/step - loss: 1.2412 - accuracy: 0.5903
Test loss 1.2411900758743286
Test accuracy 0.5903381705284119


#### **Conclusiones:**

Dado que el modelo **"LetNet_07"** alcanza el rendimiento buscado se toma como válido.

## Transfer learning

While designing your own network might produce some nice results, it is generally better to transfer the knowledge available in a pre-trained network. This not only can produce better results, but also saves a lot of design time. The [Keras Applications](https://keras.io/api/applications/) module contains several network designs ready to use. For instance, to exploit the famous VGG16 network we do

In [ ]:
from tensorflow.keras.applications import VGG16  # cargamos la red VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

# "include_top=False" parámetro para que traiga sólo las capas convolucionales (capas de clasificación no)

58900480/58889256 [==============================] - 1s 0us/step


By default all Keras Applications networks are preloaded with the weights that were obtained from training the network over the [ImageNet dataset](http://www.image-net.org/). To adapt the network to our problem we need to specify the shape of our input images, and also remove the output layers (top) of the original network, since we have a different number of classes.

Now, how do we do transfer learning over this network? We will show here how to implement the bottleneck features strategy. First, we will mark the VGG16 model as **non-trainable**, so that the weights remain frozen

In [ ]:
vgg16_model.trainable = False    # congelamos entrenamiento de capa convolucional ("FEATURES")

Now we will build a neural network that includes the VGG16 model as one of its "layers". Since the VGG16 was trained with an specific way of normalizing the images, we will need to normalize our images in the same way. Conveniently, Keras also provides a function for doing VGG16-style normalization.

In [ ]:
from keras.applications.vgg16 import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red VGG16

We can try this with some image ir our dataset

In [ ]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[137.83594 185.83594 234.83594]
  [142.92969 190.92969 238.92969]
  [148.9375  196.9375  244.9375 ]]

 [[151.30469 198.30469 244.30469]
  [145.375   192.625   241.     ]
  [246.3125  251.57812 255.     ]]

 [[191.34375 220.46094 250.38281]
  [128.00781 150.84375 207.39844]
  [ 80.92969 108.42969 173.95312]]]
After normalizing: [[[130.89694   69.05694   14.155937]
  [134.99069   74.15069   19.249687]
  [140.9985    80.1585    25.2575  ]]

 [[140.36569   81.52569   27.624687]
  [137.061     75.846     21.695   ]
  [151.061    134.79913  122.6325  ]]

 [[146.44382  103.68194   67.66375 ]
  [103.459435  34.06475    4.327812]
  [ 70.01412   -8.349312 -42.750313]]]


The normalization required by VGG16 involves swapping the order of color channels (RGB -> BGR) and substracting the mean values over the ImageNet dataset for each color channel separately. Fortunately the `preprocess_input` function does all the work for us. Furthermore, we can plug this function as the first layer of our network, similarly to the `Rescaling` we used before. We can do this with a `Lambda` layer, which allows building a layer out of any (differentiable!) function. Let's start our model with this layer.

In [ ]:
from keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))  # 1ª capa que preprocesa las imágenes según necesita la red VGG16

# "Lambda()" funcción de KERAS que nos permite introducir cualquier función de la librería TensorFlow en una capa de la red

After this, we can add the whole VGG16 network as a layer, and our custom trainable layers after it. Note this is an overly simple model with some mistakes introduced; a real transfer learning network would have a better design.

In [ ]:
model.add(vgg16_model)  # introduzco red VGG16 entera como si fuera una capa
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_75 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_158 (Dense)            (None, 6)                 49158     
Total params: 14,763,846
Trainable params: 49,158
Non-trainable params: 14,714,688
_________________________________________________________________


Notice how in the model summary we can see the whole network has millions of parameters, but since we have frozen the VGG16 part, only a few thousand parameters will be trained: those in the Dense layer.

We can now compile and train this model in the usual way

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

### **"VGG16"**



In [ ]:
import tensorflow as tf

train_dataset128, val_dataset128, test_dataset128 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset128, tf.data.Dataset)
assert isinstance(val_dataset128, tf.data.Dataset)
assert isinstance(test_dataset128, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


#### **"VGG16_01" (RED NEURONAL CONVOLUCIONAL)**



In [76]:
from tensorflow.keras.applications import VGG16  # cargamos la red VGG16
from keras.applications.vgg16 import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red VGG16

image_size = 128

vgg16_model_01 = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

# "include_top=False" parámetro para que traiga sólo las capas convolucionales (capas de clasificación no)

In [77]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "vgg16_01"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.layers import Lambda
from keras.models import Sequential

vgg16_model_01.trainable = False

image_size=128

img_rows = 128
img_cols = 128
kernel_size = 3 # Size of the kernel for the convolution layers
pool_size = 2 # Size of the pooling region for the pooling layers

vgg16_01 = Sequential()

vgg16_01.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

vgg16_01.add(RandomFlip(mode="horizontal"))
#vgg16_01.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
vgg16_01.add(RandomRotation(factor=(-0.2,0.2)))
#vgg16_01.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#vgg16_01.add(RandomCrop(height=85, width=85))
vgg16_01.add(RandomContrast(factor=0.1))

#vgg16_01.add(Rescaling(scale=1.0/255.0, input_shape=(image_size, image_size, 3)))

#vgg16_01.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

vgg16_01.add(vgg16_model_01) 



In [78]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#vgg16_01.add(BatchNormalization())
vgg16_01.add(GlobalAveragePooling2D())
#vgg16_01.add(Flatten(name="flatten"))
#vgg16_01.add(Dense(512, activation="relu"))
#vgg16_01.add(Dropout(0.25))
#vgg16_01.add(Dense(512, activation="relu"))
#vgg16_01.add(Dense(256, activation="relu"))
vgg16_01.add(Dropout(0.2))

vgg16_01.add(Dense(128, activation="relu"))
#vgg16_01.add(Dropout(0.2))

vgg16_01.add(Dense(6, activation="softmax"))

In [79]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
vgg16_01.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
from keras.callbacks import EarlyStopping

vgg16_01.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    #batch_size=256, # Batch size for the optimizer algorithm
    epochs=50, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/50
191/191 - 12s - loss: 2.3666 - accuracy: 0.4850 - val_loss: 1.1047 - val_accuracy: 0.6110
Epoch 2/50
191/191 - 8s - loss: 1.1778 - accuracy: 0.5814 - val_loss: 0.9409 - val_accuracy: 0.6679
Epoch 3/50
191/191 - 9s - loss: 1.0094 - accuracy: 0.6402 - val_loss: 0.8680 - val_accuracy: 0.6983
Epoch 4/50
191/191 - 8s - loss: 0.9438 - accuracy: 0.6508 - val_loss: 0.8057 - val_accuracy: 0.7135
Epoch 5/50
191/191 - 8s - loss: 0.8931 - accuracy: 0.6621 - val_loss: 0.8127 - val_accuracy: 0.7135
Epoch 6/50
191/191 - 8s - loss: 0.8535 - accuracy: 0.6866 - val_loss: 0.7779 - val_accuracy: 0.7396
Epoch 7/50
191/191 - 8s - loss: 0.8247 - accuracy: 0.6929 - val_loss: 0.7843 - val_accuracy: 0.7277
Epoch 8/50
191/191 - 8s - loss: 0.8007 - accuracy: 0.7001 - val_loss: 0.7422 - val_accuracy: 0.7453
Epoch 9/50
191/191 - 8s - loss: 0.7826 - accuracy: 0.7024 - val_loss: 0.7101 - val_accuracy: 0.7543
Epoch 10/50
191/191 - 8s - loss: 0.7692 - accuracy: 0.7142 - val_loss: 0.7191 - val_accuracy: 0.750

In [81]:
probs = vgg16_01.predict(test_dataset128)

In [82]:
probs

array([[8.28458928e-04, 9.67403830e-05, 8.53784800e-01, 7.50848930e-03,
        1.21925779e-01, 1.58558711e-02],
       [6.71644375e-05, 4.02063997e-05, 1.00918836e-03, 6.86462081e-05,
        1.14158483e-03, 9.97673213e-01],
       [6.48988411e-02, 2.26989463e-01, 4.43399340e-01, 2.00094700e-01,
        2.39927880e-03, 6.22183681e-02],
       ...,
       [1.99622336e-05, 9.90576863e-01, 6.76888926e-03, 5.33143175e-04,
        2.09718826e-03, 3.95471125e-06],
       [1.07907504e-01, 2.84180492e-02, 4.21103477e-01, 2.51666158e-01,
        3.72622460e-02, 1.53642550e-01],
       [3.78257856e-02, 3.07839885e-02, 6.51563555e-02, 1.34852044e-02,
        1.70890152e-01, 6.81858540e-01]], dtype=float32)

In [83]:
score = vgg16_01.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 2s 31ms/step - loss: 0.7092 - accuracy: 0.7498
Test loss 0.7092249989509583
Test accuracy 0.7497584819793701


#### **"VGG16_02" (RED NEURONAL CONVOLUCIONAL)**



In [84]:
from tensorflow.keras.applications import VGG16  # cargamos la red VGG16
from keras.applications.vgg16 import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red VGG16

image_size = 128

vgg16_model_02 = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

# "include_top=False" parámetro para que traiga sólo las capas convolucionales (capas de clasificación no)

In [85]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "vgg16_02"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.layers import Lambda
from keras.models import Sequential

vgg16_model_02.trainable = False

image_size=128

vgg16_02 = Sequential()

vgg16_02.add(RandomFlip(mode="horizontal"))
#vgg16_02.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
vgg16_02.add(RandomRotation(factor=(-0.1,0.1)))
#vgg16_02.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#vgg16_02.add(RandomCrop(height=64, width=64))
vgg16_02.add(RandomContrast(factor=0.1))

#vgg16_02.add(Resizing(image_size, image_size))

vgg16_02.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

vgg16_02.add(vgg16_model_02)

vgg16_02.add(BatchNormalization())


In [86]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

vgg16_02.add(GlobalAveragePooling2D())
vgg16_02.add(Dense(4096, activation="relu"))
vgg16_02.add(Dropout(0.825))
vgg16_02.add(Dense(4096, activation="relu"))
vgg16_02.add(Dropout(0.825))
vgg16_02.add(Dense(6, activation="softmax"))

In [87]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
vgg16_02.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [88]:
from keras.callbacks import EarlyStopping

vgg16_02.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=1024, # Batch size for the optimizer algorithm
    epochs=40, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/40
191/191 - 10s - loss: 1.7261 - accuracy: 0.5035 - val_loss: 0.8830 - val_accuracy: 0.6964
Epoch 2/40
191/191 - 8s - loss: 1.2449 - accuracy: 0.6062 - val_loss: 0.7862 - val_accuracy: 0.7073
Epoch 3/40
191/191 - 8s - loss: 1.0592 - accuracy: 0.6388 - val_loss: 0.7886 - val_accuracy: 0.7253
Epoch 4/40
191/191 - 8s - loss: 1.0653 - accuracy: 0.6542 - val_loss: 0.8466 - val_accuracy: 0.7068
Epoch 5/40
191/191 - 8s - loss: 1.0321 - accuracy: 0.6495 - val_loss: 0.7893 - val_accuracy: 0.7234
Epoch 6/40
191/191 - 8s - loss: 1.0222 - accuracy: 0.6641 - val_loss: 0.7797 - val_accuracy: 0.7315
Epoch 7/40
191/191 - 8s - loss: 0.9820 - accuracy: 0.6717 - val_loss: 0.7628 - val_accuracy: 0.7353
Epoch 8/40
191/191 - 8s - loss: 1.0197 - accuracy: 0.6725 - val_loss: 0.7435 - val_accuracy: 0.7467
Epoch 9/40
191/191 - 8s - loss: 1.0087 - accuracy: 0.6628 - val_loss: 0.7476 - val_accuracy: 0.7519
Epoch 10/40
191/191 - 8s - loss: 0.9587 - accuracy: 0.6891 - val_loss: 0.7592 - val_accuracy: 0.741

In [89]:
probs = vgg16_02.predict(test_dataset128)

In [90]:
probs

array([[4.81734090e-02, 3.20767947e-02, 6.72657669e-01, 1.34000704e-02,
        4.10379879e-02, 1.92654103e-01],
       [6.20243690e-10, 5.56742109e-12, 3.86946281e-10, 4.46189301e-11,
        1.00000000e+00, 1.44314088e-10],
       [3.21519971e-02, 2.92022829e-03, 9.80839878e-02, 1.20438579e-02,
        1.60197243e-02, 8.38780165e-01],
       ...,
       [3.53159768e-09, 3.02580450e-09, 1.00000000e+00, 8.53506016e-11,
        4.94172314e-10, 1.31170985e-08],
       [1.08521635e-05, 6.36636230e-07, 9.99980450e-01, 5.24246104e-08,
        4.61895198e-08, 8.03822149e-06],
       [1.41757322e-04, 2.28704312e-06, 2.46549072e-03, 5.06732213e-05,
        3.31269730e-05, 9.97306585e-01]], dtype=float32)

In [91]:
score = vgg16_02.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 2s 30ms/step - loss: 0.7251 - accuracy: 0.7845
Test loss 0.725104570388794
Test accuracy 0.7845410704612732


#### **Conclusiones:**

Dado que el modelo **"VGG6_02"** alcanza el rendimiento buscado se toma como válido.

## **PRO**

### **"Xception"**



In [16]:
import tensorflow as tf

train_dataset128, val_dataset128, test_dataset128 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=32)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset128, tf.data.Dataset)
assert isinstance(val_dataset128, tf.data.Dataset)
assert isinstance(test_dataset128, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [17]:
import tensorflow as tf

train_dataset256, val_dataset256, test_dataset256 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=256, batch_size=32)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset256, tf.data.Dataset)
assert isinstance(val_dataset256, tf.data.Dataset)
assert isinstance(test_dataset256, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


In [25]:
import tensorflow as tf

train_dataset299, val_dataset299, test_dataset299 = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=299, batch_size=32)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset299, tf.data.Dataset)
assert isinstance(val_dataset299, tf.data.Dataset)
assert isinstance(test_dataset299, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


#### **"Xception_01" (RED NEURONAL CONVOLUCIONAL)** 



In [92]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_01 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [93]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_01"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_01.trainable = False

image_size=128

Xception_01 = Sequential()


#Xception_01.add(RandomFlip(mode="horizontal"))
#Xception_01.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_01.add(RandomRotation(factor=(-0.2,0.2)))
#Xception_01.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_01.add(RandomCrop(height=64, width=64))
#Xception_01.add(RandomContrast(factor=0.2))

#Xception_01.add(Resizing(image_size, image_size))

Xception_01.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_01.add(Xception_model_01)

#Xception_01.add(BatchNormalization())


In [94]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

Xception_01.add(GlobalAveragePooling2D())
Xception_01.add(Flatten(name="flatten"))
Xception_01.add(Dense(526, activation="relu"))
Xception_01.add(Dropout(0.825))
Xception_01.add(Dense(258, activation="relu"))
#Xception_01.add(Dropout(0.4))
Xception_01.add(Dense(6, activation="softmax"))

In [95]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
Xception_01.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_01.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 29s - loss: 1.4640 - accuracy: 0.4727 - val_loss: 1.0142 - val_accuracy: 0.6380
Epoch 2/20
191/191 - 9s - loss: 1.2034 - accuracy: 0.5543 - val_loss: 0.9371 - val_accuracy: 0.6969
Epoch 3/20
191/191 - 9s - loss: 1.1426 - accuracy: 0.5893 - val_loss: 0.9555 - val_accuracy: 0.6917
Epoch 4/20
191/191 - 9s - loss: 1.0914 - accuracy: 0.6092 - val_loss: 0.8790 - val_accuracy: 0.7135
Epoch 5/20
191/191 - 9s - loss: 1.0253 - accuracy: 0.6287 - val_loss: 0.9133 - val_accuracy: 0.7064
Epoch 6/20
191/191 - 9s - loss: 0.9971 - accuracy: 0.6412 - val_loss: 0.8839 - val_accuracy: 0.7249
Epoch 7/20
191/191 - 9s - loss: 0.9697 - accuracy: 0.6565 - val_loss: 0.8861 - val_accuracy: 0.7287
Epoch 8/20
191/191 - 9s - loss: 0.9218 - accuracy: 0.6749 - val_loss: 0.8989 - val_accuracy: 0.7173
Epoch 9/20
191/191 - 9s - loss: 0.9144 - accuracy: 0.6717 - val_loss: 0.8934 - val_accuracy: 0.7277
Epoch 10/20
191/191 - 9s - loss: 0.8718 - accuracy: 0.6883 - val_loss: 0.8827 - val_accuracy: 0.736

In [ ]:
probs = Xception_01.predict(test_dataset128)

In [ ]:
probs

array([[2.5656180e-02, 1.9363075e-01, 5.4848373e-01, 1.7532441e-01,
        2.4570683e-02, 3.2334235e-02],
       [1.0227151e-01, 1.2386827e-02, 3.2749197e-01, 6.3645318e-02,
        4.1547161e-02, 4.5265725e-01],
       [7.2836839e-03, 4.1211133e-05, 6.8927230e-04, 9.1391848e-04,
        9.7816968e-01, 1.2902213e-02],
       ...,
       [2.2241887e-02, 4.6178445e-01, 2.4805592e-01, 1.5210345e-01,
        1.0232210e-01, 1.3492165e-02],
       [3.9172879e-01, 1.0286564e-02, 8.4050529e-02, 3.6675769e-01,
        1.0063163e-01, 4.6544783e-02],
       [1.0523128e-01, 8.5874749e-03, 4.0059235e-02, 6.8522340e-01,
        9.3820438e-02, 6.7078181e-02]], dtype=float32)

In [ ]:
score = Xception_01.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 40ms/step - loss: 0.8247 - accuracy: 0.7430
Test loss 0.8246902227401733
Test accuracy 0.7429951429367065


#### **"Xception_02" (RED NEURONAL CONVOLUCIONAL)** 



In [199]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_02 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [200]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_02"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_02.trainable = False

image_size=128

Xception_02 = Sequential()

Xception_02.add(RandomFlip(mode="horizontal"))
#Xception_02.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_02.add(RandomRotation(factor=(-0.3,0.3)))
#Xception_02.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_02.add(RandomCrop(height=64, width=64))
Xception_02.add(RandomContrast(factor=0.3))

#Xception_02.add(Resizing(image_size, image_size))

Xception_02.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_02.add(Xception_model_02)

#Xception_02.add(BatchNormalization())


In [201]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

Xception_02.add(GlobalAveragePooling2D())
Xception_02.add(Flatten(name="flatten"))
Xception_02.add(Dense(124, activation="relu"))
Xception_02.add(Dropout(0.325))
Xception_02.add(Dense(6, activation="softmax"))

In [202]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_02.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [203]:
from keras.callbacks import EarlyStopping

Xception_02.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 15s - loss: 1.1323 - accuracy: 0.5894 - val_loss: 0.8382 - val_accuracy: 0.7016
Epoch 2/20
191/191 - 9s - loss: 0.9456 - accuracy: 0.6592 - val_loss: 0.7996 - val_accuracy: 0.7187
Epoch 3/20
191/191 - 10s - loss: 0.8857 - accuracy: 0.6748 - val_loss: 0.7670 - val_accuracy: 0.7306
Epoch 4/20
191/191 - 9s - loss: 0.8542 - accuracy: 0.6866 - val_loss: 0.7891 - val_accuracy: 0.7296
Epoch 5/20
191/191 - 9s - loss: 0.8148 - accuracy: 0.7055 - val_loss: 0.7654 - val_accuracy: 0.7377
Epoch 6/20
191/191 - 9s - loss: 0.7924 - accuracy: 0.7134 - val_loss: 0.7314 - val_accuracy: 0.7514
Epoch 7/20
191/191 - 9s - loss: 0.7783 - accuracy: 0.7106 - val_loss: 0.7738 - val_accuracy: 0.7296
Epoch 8/20
191/191 - 9s - loss: 0.7638 - accuracy: 0.7220 - val_loss: 0.7368 - val_accuracy: 0.7500
Epoch 9/20
191/191 - 9s - loss: 0.7668 - accuracy: 0.7226 - val_loss: 0.7130 - val_accuracy: 0.7519
Epoch 10/20
191/191 - 9s - loss: 0.7172 - accuracy: 0.7399 - val_loss: 0.7479 - val_accuracy: 0.74

In [204]:
probs = Xception_02.predict(test_dataset128)

In [205]:
probs

array([[2.66033024e-01, 1.48055179e-03, 1.12613805e-01, 1.14134848e-01,
        2.79622450e-02, 4.77775604e-01],
       [1.41100225e-03, 9.28487718e-01, 4.08343039e-02, 2.42909845e-02,
        3.65057169e-03, 1.32549438e-03],
       [8.64221990e-01, 1.20460914e-04, 1.58549671e-03, 2.39848834e-03,
        1.31514564e-01, 1.58981115e-04],
       ...,
       [5.29935723e-03, 1.26845658e-01, 7.09381700e-01, 6.54348347e-04,
        1.57780439e-01, 3.85560525e-05],
       [8.26327085e-01, 1.08056985e-04, 1.27981221e-02, 1.33469164e-01,
        1.52346864e-03, 2.57741306e-02],
       [2.29764101e-03, 1.76291942e-05, 9.95881081e-01, 1.16311142e-03,
        6.09282961e-06, 6.34524855e-04]], dtype=float32)

In [206]:
score = Xception_02.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 37ms/step - loss: 0.6918 - accuracy: 0.7517
Test loss 0.6918388605117798
Test accuracy 0.7516908049583435


#### **"Xception_03" (RED NEURONAL CONVOLUCIONAL)** 



In [100]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_03 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [101]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_03"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_03.trainable = False

image_size=128

Xception_03 = Sequential()

Xception_03.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Xception_03.add(RandomFlip(mode="horizontal"))
#Xception_03.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_03.add(RandomRotation(factor=(-0.3,0.3)))
#Xception_03.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_03.add(RandomCrop(height=64, width=64))
Xception_03.add(RandomContrast(factor=0.3))

#Xception_03.add(Resizing(image_size, image_size))

#Xception_03.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_03.add(Xception_model_03)

#Xception_03.add(BatchNormalization())


In [102]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_03.add(GlobalAveragePooling2D())
#Xception_03.add(Flatten(name="flatten"))
#Xception_03.add(Dense(124, activation="relu"))
#Xception_03.add(Dropout(0.325))

Xception_03.add(GlobalAveragePooling2D())
Xception_03.add(Dropout(0.2))
Xception_03.add(Dense(128, activation="relu"))
Xception_03.add(Dense(6, activation="softmax"))

In [103]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_03.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_03.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 13s - loss: 1.0462 - accuracy: 0.6125 - val_loss: 0.8124 - val_accuracy: 0.7125
Epoch 2/20
191/191 - 9s - loss: 0.8858 - accuracy: 0.6756 - val_loss: 0.7744 - val_accuracy: 0.7230
Epoch 3/20
191/191 - 9s - loss: 0.8319 - accuracy: 0.6970 - val_loss: 0.7967 - val_accuracy: 0.7192
Epoch 4/20
191/191 - 9s - loss: 0.8005 - accuracy: 0.7042 - val_loss: 0.7546 - val_accuracy: 0.7358
Epoch 5/20
191/191 - 9s - loss: 0.7814 - accuracy: 0.7139 - val_loss: 0.7426 - val_accuracy: 0.7315
Epoch 6/20
191/191 - 9s - loss: 0.7424 - accuracy: 0.7261 - val_loss: 0.7626 - val_accuracy: 0.7353
Epoch 7/20
191/191 - 9s - loss: 0.7238 - accuracy: 0.7425 - val_loss: 0.7292 - val_accuracy: 0.7438
Epoch 8/20
191/191 - 9s - loss: 0.7118 - accuracy: 0.7353 - val_loss: 0.7393 - val_accuracy: 0.7495
Epoch 9/20
191/191 - 9s - loss: 0.6902 - accuracy: 0.7461 - val_loss: 0.7134 - val_accuracy: 0.7619
Epoch 10/20
191/191 - 9s - loss: 0.6584 - accuracy: 0.7558 - val_loss: 0.7301 - val_accuracy: 0.744

In [ ]:
probs = Xception_03.predict(test_dataset128)

In [ ]:
probs

array([[1.8478826e-02, 9.4686824e-01, 2.9692622e-02, 4.3826941e-03,
        3.5677222e-04, 2.2077093e-04],
       [2.5495762e-02, 3.6125355e-03, 5.6086677e-01, 4.1423100e-03,
        6.6999689e-04, 4.0521267e-01],
       [2.8144272e-02, 8.6918083e-04, 9.9270009e-03, 7.5634629e-02,
        7.0513922e-01, 1.8028568e-01],
       ...,
       [1.6862992e-03, 3.3198326e-04, 5.3051766e-03, 9.9083227e-01,
        7.1992242e-04, 1.1243555e-03],
       [2.9328457e-01, 4.1125230e-02, 2.0978114e-01, 4.3635428e-01,
        1.7068150e-02, 2.3867500e-03],
       [9.9407578e-01, 2.0522374e-04, 1.6179309e-03, 3.5933473e-03,
        3.6496992e-04, 1.4268514e-04]], dtype=float32)

In [ ]:
score = Xception_03.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 39ms/step - loss: 0.7067 - accuracy: 0.7483
Test loss 0.7067177295684814
Test accuracy 0.7483091950416565


#### **"Xception_04" (RED NEURONAL CONVOLUCIONAL)** 



In [104]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_04 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [105]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_04"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_04.trainable = False

image_size=128

Xception_04 = Sequential()

Xception_04.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Xception_04.add(RandomFlip(mode="horizontal"))
#Xception_04.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_04.add(RandomRotation(factor=(-0.3,0.3)))
#Xception_04.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_04.add(RandomCrop(height=64, width=64))
Xception_04.add(RandomContrast(factor=0.3))

#Xception_04.add(Resizing(image_size, image_size))

#Xception_04.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_04.add(Xception_model_04)

#Xception_04.add(BatchNormalization())


In [106]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_04.add(GlobalAveragePooling2D())
#Xception_04.add(Flatten(name="flatten"))
#Xception_04.add(Dense(124, activation="relu"))
#Xception_04.add(Dropout(0.325))

Xception_04.add(GlobalAveragePooling2D())
Xception_04.add(Dropout(0.4))
Xception_04.add(Dense(128, activation="relu"))
Xception_04.add(Dense(6, activation="softmax"))

In [107]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_04.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_04.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 13s - loss: 1.1181 - accuracy: 0.5881 - val_loss: 0.8886 - val_accuracy: 0.6803
Epoch 2/20
191/191 - 9s - loss: 0.9348 - accuracy: 0.6488 - val_loss: 0.7898 - val_accuracy: 0.7306
Epoch 3/20
191/191 - 9s - loss: 0.8755 - accuracy: 0.6756 - val_loss: 0.7691 - val_accuracy: 0.7339
Epoch 4/20
191/191 - 9s - loss: 0.8458 - accuracy: 0.6888 - val_loss: 0.7809 - val_accuracy: 0.7258
Epoch 5/20
191/191 - 9s - loss: 0.8186 - accuracy: 0.7011 - val_loss: 0.7475 - val_accuracy: 0.7434
Epoch 6/20
191/191 - 9s - loss: 0.8207 - accuracy: 0.6989 - val_loss: 0.7664 - val_accuracy: 0.7282
Epoch 7/20
191/191 - 9s - loss: 0.7735 - accuracy: 0.7169 - val_loss: 0.7375 - val_accuracy: 0.7367
Epoch 8/20
191/191 - 9s - loss: 0.7873 - accuracy: 0.7095 - val_loss: 0.7347 - val_accuracy: 0.7472
Epoch 9/20
191/191 - 9s - loss: 0.7593 - accuracy: 0.7200 - val_loss: 0.7171 - val_accuracy: 0.7438
Epoch 10/20
191/191 - 9s - loss: 0.7422 - accuracy: 0.7308 - val_loss: 0.7178 - val_accuracy: 0.746

In [ ]:
probs = Xception_04.predict(test_dataset128)

In [ ]:
probs

array([[8.2047097e-02, 9.1225374e-06, 4.2449083e-02, 4.1655727e-02,
        2.0746699e-02, 8.1309229e-01],
       [2.4724575e-02, 6.6795328e-04, 1.4262925e-02, 9.5509499e-01,
        4.1303271e-03, 1.1191954e-03],
       [2.2120936e-01, 3.9430469e-02, 8.7776214e-02, 1.3250618e-01,
        5.0254405e-01, 1.6533744e-02],
       ...,
       [4.1581439e-03, 5.1826117e-07, 4.3810820e-03, 6.4870506e-03,
        6.5566582e-04, 9.8431754e-01],
       [1.5169637e-03, 9.7629434e-01, 1.9815119e-02, 1.5670460e-04,
        2.1615461e-03, 5.5314977e-05],
       [2.2778916e-03, 4.0777776e-02, 1.6461903e-01, 1.1792609e-03,
        1.7149830e-02, 7.7399617e-01]], dtype=float32)

In [ ]:
score = Xception_04.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 41ms/step - loss: 0.7059 - accuracy: 0.7498
Test loss 0.7059474587440491
Test accuracy 0.7497584819793701


#### **"Xception_05" (RED NEURONAL CONVOLUCIONAL)** 



In [108]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_05 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [109]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_05"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_05.trainable = False

image_size=128

Xception_05 = Sequential()

Xception_05.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Xception_05.add(RandomFlip(mode="horizontal"))
#Xception_05.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_05.add(RandomRotation(factor=(-0.3,0.3)))
#Xception_05.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_05.add(RandomCrop(height=64, width=64))
Xception_05.add(RandomContrast(factor=0.3))

#Xception_05.add(Resizing(image_size, image_size))

#Xception_05.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_05.add(BatchNormalization())

Xception_05.add(Xception_model_05)

#Xception_05.add(BatchNormalization())


In [110]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_05.add(GlobalAveragePooling2D())
#Xception_05.add(Flatten(name="flatten"))
#Xception_05.add(Dense(124, activation="relu"))
#Xception_05.add(Dropout(0.325))

Xception_05.add(GlobalAveragePooling2D())
Xception_05.add(Dropout(0.4))
Xception_05.add(Dense(10240, activation="relu"))
#Xception_05.add(Dropout(0.4))
#Xception_05.add(Dense(4096, activation="relu"))
#Xception_05.add(Dropout(0.4))
Xception_05.add(Dense(6, activation="softmax"))

In [111]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_05.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_05.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 23s - loss: 1.4427 - accuracy: 0.5564 - val_loss: 0.9854 - val_accuracy: 0.6338
Epoch 2/20
191/191 - 18s - loss: 1.0069 - accuracy: 0.6235 - val_loss: 0.9313 - val_accuracy: 0.6556
Epoch 3/20
191/191 - 18s - loss: 0.9521 - accuracy: 0.6501 - val_loss: 0.8317 - val_accuracy: 0.7144
Epoch 4/20
191/191 - 18s - loss: 0.9048 - accuracy: 0.6672 - val_loss: 0.7950 - val_accuracy: 0.7158
Epoch 5/20
191/191 - 18s - loss: 0.8633 - accuracy: 0.6848 - val_loss: 0.8032 - val_accuracy: 0.7192
Epoch 6/20
191/191 - 18s - loss: 0.8313 - accuracy: 0.6907 - val_loss: 0.7563 - val_accuracy: 0.7348
Epoch 7/20
191/191 - 18s - loss: 0.8225 - accuracy: 0.7047 - val_loss: 0.7951 - val_accuracy: 0.7092
Epoch 8/20
191/191 - 18s - loss: 0.7617 - accuracy: 0.7203 - val_loss: 0.7691 - val_accuracy: 0.7230
Epoch 9/20
191/191 - 18s - loss: 0.7388 - accuracy: 0.7361 - val_loss: 0.7377 - val_accuracy: 0.7462
Epoch 10/20
191/191 - 18s - loss: 0.7379 - accuracy: 0.7284 - val_loss: 0.7715 - val_accura

In [ ]:
probs = Xception_05.predict(test_dataset128)

In [ ]:
probs

array([[3.00627999e-05, 1.07990138e-01, 5.27922630e-01, 3.63992870e-01,
        4.90229613e-05, 1.53034234e-05],
       [1.63527020e-05, 4.28126477e-05, 3.61084181e-04, 9.99550641e-01,
        6.61909053e-06, 2.24610267e-05],
       [1.35537043e-01, 2.37608892e-05, 8.63934338e-01, 4.91347782e-05,
        7.06752826e-06, 4.48691571e-04],
       ...,
       [5.18840551e-03, 3.37794409e-05, 3.29510751e-03, 1.81082869e-05,
        9.91400540e-01, 6.40907383e-05],
       [9.08057094e-02, 6.72620197e-04, 9.90287662e-02, 5.85549064e-02,
        1.23588935e-01, 6.27349019e-01],
       [9.99985337e-01, 1.50487990e-11, 1.32580344e-05, 1.25287431e-06,
        4.69386983e-08, 7.55105205e-08]], dtype=float32)

In [ ]:
score = Xception_05.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 39ms/step - loss: 0.7389 - accuracy: 0.7464
Test loss 0.7388555407524109
Test accuracy 0.7463768124580383


#### **"Xception_06" (RED NEURONAL CONVOLUCIONAL)** 



In [112]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_06 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [113]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_06"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_06.trainable = False

image_size=128

Xception_06 = Sequential()

Xception_06.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Xception_06.add(RandomFlip(mode="horizontal"))
#Xception_06.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_06.add(RandomRotation(factor=(-0.3,0.3)))
#Xception_06.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_06.add(RandomCrop(height=64, width=64))
Xception_06.add(RandomContrast(factor=0.3))

#Xception_06.add(Resizing(image_size, image_size))

#Xception_06.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_06.add(BatchNormalization())

Xception_06.add(Xception_model_06)

#Xception_06.add(BatchNormalization())


In [114]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_06.add(GlobalAveragePooling2D())
#Xception_06.add(Flatten(name="flatten"))
#Xception_06.add(Dense(124, activation="relu"))
#Xception_06.add(Dropout(0.325))

Xception_06.add(GlobalAveragePooling2D())
Xception_06.add(Dropout(0.425))
Xception_06.add(Dense(4096, activation="relu"))
#Xception_06.add(Dropout(0.4))
#Xception_06.add(Dense(4096, activation="relu"))
#Xception_06.add(Dropout(0.4))
Xception_06.add(Dense(6, activation="softmax"))

In [115]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_06.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_06.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 22s - loss: 1.3089 - accuracy: 0.5554 - val_loss: 0.9781 - val_accuracy: 0.6395
Epoch 2/20
191/191 - 17s - loss: 1.0271 - accuracy: 0.6143 - val_loss: 0.8907 - val_accuracy: 0.6736
Epoch 3/20
191/191 - 17s - loss: 0.9436 - accuracy: 0.6516 - val_loss: 0.8522 - val_accuracy: 0.6969
Epoch 4/20
191/191 - 17s - loss: 0.9058 - accuracy: 0.6677 - val_loss: 0.8445 - val_accuracy: 0.7002
Epoch 5/20
191/191 - 17s - loss: 0.8736 - accuracy: 0.6802 - val_loss: 0.8403 - val_accuracy: 0.6992
Epoch 6/20
191/191 - 17s - loss: 0.8388 - accuracy: 0.6917 - val_loss: 0.8228 - val_accuracy: 0.7102
Epoch 7/20
191/191 - 17s - loss: 0.8186 - accuracy: 0.6991 - val_loss: 0.7572 - val_accuracy: 0.7367
Epoch 8/20
191/191 - 17s - loss: 0.7960 - accuracy: 0.6978 - val_loss: 0.8022 - val_accuracy: 0.7149
Epoch 9/20
191/191 - 17s - loss: 0.7563 - accuracy: 0.7238 - val_loss: 0.7610 - val_accuracy: 0.7339
Epoch 10/20
191/191 - 17s - loss: 0.7610 - accuracy: 0.7179 - val_loss: 0.7570 - val_accura

In [ ]:
probs = Xception_06.predict(test_dataset128)

In [ ]:
probs

array([[2.21622381e-02, 1.86057448e-01, 7.66516566e-01, 1.55370180e-02,
        5.21354517e-03, 4.51317988e-03],
       [1.04911085e-02, 1.20130882e-01, 4.20834839e-01, 4.32155073e-01,
        1.22003406e-02, 4.18765983e-03],
       [7.88803935e-01, 2.38671655e-05, 6.71056448e-04, 1.89846812e-03,
        2.03412652e-01, 5.19000692e-03],
       ...,
       [4.50996886e-05, 3.24213170e-02, 9.67521548e-01, 2.42718806e-06,
        8.37386597e-06, 1.31022159e-06],
       [1.29110576e-03, 7.86271021e-02, 8.91767859e-01, 1.70600452e-02,
        2.04129890e-03, 9.21260938e-03],
       [3.90130794e-03, 8.38731008e-04, 3.68128978e-02, 4.80816874e-04,
        1.15844328e-03, 9.56807852e-01]], dtype=float32)

In [ ]:
score = Xception_06.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 39ms/step - loss: 0.7215 - accuracy: 0.7517
Test loss 0.7214577198028564
Test accuracy 0.7516908049583435


#### **"Xception_07" (RED NEURONAL CONVOLUCIONAL)** 



In [116]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_07 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [117]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_07"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_07.trainable = False

image_size=128

Xception_07 = Sequential()

Xception_07.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Xception_07.add(RandomFlip(mode="horizontal"))
#Xception_07.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_07.add(RandomRotation(factor=(-0.3,0.3)))
#Xception_07.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_07.add(RandomCrop(height=64, width=64))
Xception_07.add(RandomContrast(factor=0.35))

#Xception_07.add(Resizing(image_size, image_size))

#Xception_07.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_07.add(BatchNormalization())

Xception_07.add(Xception_model_07)

#Xception_07.add(BatchNormalization())


In [118]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_07.add(GlobalAveragePooling2D())
#Xception_07.add(Flatten(name="flatten"))
#Xception_07.add(Dense(124, activation="relu"))
#Xception_07.add(Dropout(0.325))

Xception_07.add(GlobalAveragePooling2D())
Xception_07.add(Dropout(0.4))
Xception_07.add(Dense(20000, activation="relu"))
#Xception_07.add(Dropout(0.4))
#Xception_07.add(Dense(4096, activation="relu"))
#Xception_07.add(Dropout(0.4))
Xception_07.add(Dense(6, activation="softmax"))

In [119]:
# COMPILAMOS Y ENTRENAMOS EL MODELO
Xception_07.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_07.fit(
    train_dataset128, # Training data
    validation_data=val_dataset128,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 23s - loss: 1.6992 - accuracy: 0.5311 - val_loss: 1.0773 - val_accuracy: 0.6176
Epoch 2/20
191/191 - 18s - loss: 1.0222 - accuracy: 0.6220 - val_loss: 0.8899 - val_accuracy: 0.6845
Epoch 3/20
191/191 - 18s - loss: 0.9363 - accuracy: 0.6565 - val_loss: 0.8326 - val_accuracy: 0.7144
Epoch 4/20
191/191 - 18s - loss: 0.9086 - accuracy: 0.6620 - val_loss: 0.8288 - val_accuracy: 0.6988
Epoch 5/20
191/191 - 18s - loss: 0.8638 - accuracy: 0.6800 - val_loss: 0.7798 - val_accuracy: 0.7296
Epoch 6/20
191/191 - 18s - loss: 0.8410 - accuracy: 0.6810 - val_loss: 0.7791 - val_accuracy: 0.7263
Epoch 7/20
191/191 - 18s - loss: 0.8146 - accuracy: 0.6943 - val_loss: 0.7726 - val_accuracy: 0.7244
Epoch 8/20
191/191 - 18s - loss: 0.7926 - accuracy: 0.7116 - val_loss: 0.7683 - val_accuracy: 0.7329
Epoch 9/20
191/191 - 18s - loss: 0.7671 - accuracy: 0.7137 - val_loss: 0.7461 - val_accuracy: 0.7348
Epoch 10/20
191/191 - 18s - loss: 0.7370 - accuracy: 0.7294 - val_loss: 0.7898 - val_accura

In [ ]:
probs = Xception_07.predict(test_dataset128)

In [ ]:
probs

array([[3.1187331e-02, 4.1809768e-04, 2.8119015e-02, 9.2365104e-01,
        6.9186245e-03, 9.7058555e-03],
       [2.3326330e-04, 1.4647424e-05, 8.2416018e-04, 1.2806116e-04,
        4.0253748e-05, 9.9875963e-01],
       [1.7799764e-03, 8.8318489e-02, 2.7713068e-02, 8.7967795e-01,
        1.3059330e-03, 1.2046031e-03],
       ...,
       [4.3919794e-03, 1.7319683e-02, 7.0950590e-02, 1.2932039e-03,
        8.2007367e-03, 8.9784384e-01],
       [6.5933824e-02, 4.0969956e-03, 8.9316779e-01, 3.3819363e-02,
        1.0071383e-03, 1.9747640e-03],
       [1.9798519e-02, 2.0853510e-02, 8.9422983e-01, 1.3158223e-02,
        9.0952022e-03, 4.2864766e-02]], dtype=float32)

In [ ]:
score = Xception_07.evaluate(test_dataset128)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 40ms/step - loss: 0.7161 - accuracy: 0.7498
Test loss 0.7160783410072327
Test accuracy 0.7497584819793701


#### **"Xception_08" (RED NEURONAL CONVOLUCIONAL)** 



In [120]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_08 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [121]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_08"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_08.trainable = False

image_size_in=256

image_size_model=128

Xception_08 = Sequential()

Xception_08.add(Lambda(preprocess_input, input_shape=(image_size_in, image_size_in, 3)))

Xception_08.add(RandomFlip(mode="horizontal"))
#Xception_08.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_08.add(RandomRotation(factor=(-0.35,0.35)))
#Xception_08.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_08.add(RandomCrop(height=64, width=64))
Xception_08.add(RandomContrast(factor=0.35))

Xception_08.add(Resizing(image_size_model, image_size_model))

#Xception_08.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_08.add(BatchNormalization())

Xception_08.add(Xception_model_08)

#Xception_08.add(BatchNormalization())


In [122]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_08.add(GlobalAveragePooling2D())
#Xception_08.add(Flatten(name="flatten"))
#Xception_08.add(Dense(124, activation="relu"))
#Xception_08.add(Dropout(0.325))

Xception_08.add(GlobalAveragePooling2D())
Xception_08.add(Dropout(0.4))
Xception_08.add(Dense(4096, activation="relu"))
Xception_08.add(Dropout(0.4))
Xception_08.add(Dense(4096, activation="relu"))
#Xception_08.add(Dropout(0.4))
Xception_08.add(Dense(6, activation="softmax"))

In [123]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_08.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_08.fit(
    train_dataset256, # Training data
    validation_data=val_dataset256,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 24s - loss: 1.4879 - accuracy: 0.5099 - val_loss: 1.0204 - val_accuracy: 0.6257
Epoch 2/20
191/191 - 20s - loss: 1.1136 - accuracy: 0.5842 - val_loss: 0.9823 - val_accuracy: 0.6518
Epoch 3/20
191/191 - 20s - loss: 1.0256 - accuracy: 0.6246 - val_loss: 1.0276 - val_accuracy: 0.6276
Epoch 4/20
191/191 - 20s - loss: 1.0107 - accuracy: 0.6282 - val_loss: 0.8887 - val_accuracy: 0.6945
Epoch 5/20
191/191 - 20s - loss: 0.9495 - accuracy: 0.6595 - val_loss: 0.9364 - val_accuracy: 0.6665
Epoch 6/20
191/191 - 20s - loss: 0.9351 - accuracy: 0.6588 - val_loss: 0.8800 - val_accuracy: 0.6912
Epoch 7/20
191/191 - 20s - loss: 0.9168 - accuracy: 0.6624 - val_loss: 0.7963 - val_accuracy: 0.7234
Epoch 8/20
191/191 - 20s - loss: 0.9029 - accuracy: 0.6744 - val_loss: 0.8054 - val_accuracy: 0.7139
Epoch 9/20
191/191 - 20s - loss: 0.8878 - accuracy: 0.6743 - val_loss: 0.8036 - val_accuracy: 0.7225
Epoch 10/20
191/191 - 20s - loss: 0.8767 - accuracy: 0.6841 - val_loss: 0.8136 - val_accura

In [ ]:
probs = Xception_08.predict(test_dataset256)

In [ ]:
probs

array([[1.4079761e-04, 2.9214441e-03, 6.5671408e-01, 3.3376923e-01,
        6.2656025e-03, 1.8878197e-04],
       [8.9763325e-01, 2.4986803e-05, 7.9193758e-04, 9.0257592e-02,
        1.5704862e-03, 9.7216163e-03],
       [2.2637451e-01, 2.0104339e-02, 3.3889678e-01, 1.4850448e-01,
        1.1009477e-01, 1.5602517e-01],
       ...,
       [2.2778795e-03, 6.7920035e-07, 2.5282163e-04, 2.7742670e-04,
        9.0592615e-03, 9.8813194e-01],
       [3.4372300e-01, 4.9109990e-04, 1.2647092e-01, 5.0628030e-01,
        1.6728515e-02, 6.3062599e-03],
       [3.5477662e-04, 1.9524005e-07, 3.7100490e-06, 2.0562878e-05,
        9.9926931e-01, 3.5139910e-04]], dtype=float32)

In [ ]:
score = Xception_08.evaluate(test_dataset256)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 47ms/step - loss: 0.7593 - accuracy: 0.7353
Test loss 0.7593244910240173
Test accuracy 0.7352656722068787


#### **"Xception_09" (RED NEURONAL CONVOLUCIONAL)** 



In [124]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 128

Xception_model_09 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [125]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_09"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_09.trainable = False

image_size_in=256

image_size_model=128

Xception_09 = Sequential()

Xception_09.add(Lambda(preprocess_input, input_shape=(image_size_in, image_size_in, 3)))

Xception_09.add(RandomFlip(mode="horizontal"))
#Xception_09.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_09.add(RandomRotation(factor=(-0.4,0.4)))
#Xception_09.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_09.add(RandomCrop(height=64, width=64))
Xception_09.add(RandomContrast(factor=0.4))

Xception_09.add(Resizing(image_size_model, image_size_model))

#Xception_09.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_09.add(BatchNormalization())

Xception_09.add(Xception_model_09)

#Xception_09.add(BatchNormalization())


In [126]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_09.add(GlobalAveragePooling2D())
#Xception_09.add(Flatten(name="flatten"))
#Xception_09.add(Dense(124, activation="relu"))
#Xception_09.add(Dropout(0.325))

Xception_09.add(GlobalAveragePooling2D())
Xception_09.add(Dropout(0.5))
Xception_09.add(Dense(4096, activation="relu"))
Xception_09.add(Dropout(0.5))
Xception_09.add(Dense(4096, activation="relu"))
#Xception_09.add(Dropout(0.4))
Xception_09.add(Dense(6, activation="softmax"))

In [127]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_09.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

Xception_09.fit(
    train_dataset256, # Training data
    validation_data=val_dataset256,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=20, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/20
191/191 - 23s - loss: 1.5857 - accuracy: 0.4824 - val_loss: 1.0571 - val_accuracy: 0.6229
Epoch 2/20
191/191 - 19s - loss: 1.1738 - accuracy: 0.5715 - val_loss: 0.9862 - val_accuracy: 0.6414
Epoch 3/20
191/191 - 19s - loss: 1.1060 - accuracy: 0.5983 - val_loss: 0.9472 - val_accuracy: 0.6708
Epoch 4/20
191/191 - 19s - loss: 1.0597 - accuracy: 0.6084 - val_loss: 0.9131 - val_accuracy: 0.6898
Epoch 5/20
191/191 - 19s - loss: 1.0274 - accuracy: 0.6208 - val_loss: 0.9275 - val_accuracy: 0.6784
Epoch 6/20
191/191 - 19s - loss: 0.9958 - accuracy: 0.6407 - val_loss: 0.9160 - val_accuracy: 0.6660
Epoch 7/20
191/191 - 19s - loss: 0.9887 - accuracy: 0.6274 - val_loss: 0.9003 - val_accuracy: 0.6855
Epoch 8/20
191/191 - 19s - loss: 0.9720 - accuracy: 0.6447 - val_loss: 0.8642 - val_accuracy: 0.7002
Epoch 9/20
191/191 - 19s - loss: 0.9673 - accuracy: 0.6468 - val_loss: 0.8428 - val_accuracy: 0.7168
Epoch 10/20
191/191 - 19s - loss: 0.9564 - accuracy: 0.6490 - val_loss: 0.8548 - val_accura

In [ ]:
probs = Xception_09.predict(test_dataset256)

In [ ]:
probs

array([[9.5813513e-02, 2.7820815e-03, 1.2146912e-01, 3.2051552e-02,
        2.8838214e-02, 7.1904546e-01],
       [5.5781585e-01, 4.0686498e-03, 1.0557474e-01, 1.3272688e-01,
        1.4643119e-01, 5.3382616e-02],
       [4.4676349e-02, 6.7304023e-02, 1.5760207e-01, 6.9047964e-01,
        2.8783629e-02, 1.1154248e-02],
       ...,
       [5.3633563e-02, 7.5486349e-04, 3.6047988e-02, 2.3316128e-02,
        1.1174751e-02, 8.7507272e-01],
       [9.8427469e-01, 3.5312232e-11, 2.2840503e-04, 1.5496218e-02,
        1.4051924e-10, 7.4728200e-07],
       [2.4423765e-01, 7.9063801e-03, 2.0636912e-01, 2.8913486e-01,
        2.0707361e-01, 4.5278393e-02]], dtype=float32)

In [ ]:
score = Xception_09.evaluate(test_dataset256)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 3s 49ms/step - loss: 0.7605 - accuracy: 0.7329
Test loss 0.7605149745941162
Test accuracy 0.7328502535820007


#### **"Xception_10" (RED NEURONAL CONVOLUCIONAL)**



In [26]:
from tensorflow.keras.applications import Xception  # cargamos la red Xception
from keras.applications.xception import preprocess_input   # para importar función de prepocesamiento de imágenes necesario para la red Xception

image_size = 299

Xception_model_10 = Xception(include_top=False, input_shape=(image_size, image_size, 3))

In [27]:
# DEFINIMOS LA RED NEURONAL CONVOLUCIONAL "Xception_10"
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, Resizing
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomCrop, RandomTranslation, RandomRotation, RandomZoom, Rescaling, RandomContrast
from keras.layers import Lambda
from keras.models import Sequential

Xception_model_10.trainable = False

image_size_in=299

#image_size_model=128

Xception_10 = Sequential()

Xception_10.add(Lambda(preprocess_input, input_shape=(image_size_in, image_size_in, 3)))

Xception_10.add(RandomFlip(mode="horizontal"))
#Xception_10.add(RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
Xception_10.add(RandomRotation(factor=(-0.35,0.35)))
#Xception_10.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))
#Xception_10.add(RandomCrop(height=64, width=64))
Xception_10.add(RandomContrast(factor=0.35))

#Xception_10.add(Resizing(image_size_model, image_size_model))

#Xception_10.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3))) 

Xception_10.add(BatchNormalization())

Xception_10.add(Xception_model_10)

#Xception_10.add(BatchNormalization())


In [28]:
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D

#Xception_10.add(GlobalAveragePooling2D())
#Xception_10.add(Flatten(name="flatten"))
#Xception_10.add(Dense(124, activation="relu"))
#Xception_10.add(Dropout(0.325))

Xception_10.add(GlobalAveragePooling2D())
Xception_10.add(Dropout(0.45))
Xception_10.add(Dense(4096, activation="relu"))
Xception_10.add(Dropout(0.45))
Xception_10.add(Dense(4096, activation="relu"))
#Xception_10.add(Dropout(0.4))
Xception_10.add(Dense(6, activation="softmax"))

In [29]:
# COMPILAMOS Y ENTRENAMOS EL MODELO

Xception_10.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#####  **Nº de épocas: 50**

In [30]:
from keras.callbacks import EarlyStopping

Xception_10.fit(
    train_dataset299, # Training data
    validation_data=val_dataset299,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=50, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/50
191/191 - 88s - loss: 0.9814 - accuracy: 0.6636 - val_loss: 0.6091 - val_accuracy: 0.7936
Epoch 2/50
191/191 - 83s - loss: 0.7224 - accuracy: 0.7432 - val_loss: 0.5616 - val_accuracy: 0.8060
Epoch 3/50
191/191 - 83s - loss: 0.6621 - accuracy: 0.7655 - val_loss: 0.4909 - val_accuracy: 0.8207
Epoch 4/50
191/191 - 83s - loss: 0.6248 - accuracy: 0.7739 - val_loss: 0.4811 - val_accuracy: 0.8283
Epoch 5/50
191/191 - 83s - loss: 0.6010 - accuracy: 0.7828 - val_loss: 0.4481 - val_accuracy: 0.8539
Epoch 6/50
191/191 - 83s - loss: 0.5519 - accuracy: 0.8014 - val_loss: 0.4409 - val_accuracy: 0.8449
Epoch 7/50
191/191 - 83s - loss: 0.5384 - accuracy: 0.8103 - val_loss: 0.4738 - val_accuracy: 0.8245
Epoch 8/50
191/191 - 83s - loss: 0.5449 - accuracy: 0.8057 - val_loss: 0.4176 - val_accuracy: 0.8558
Epoch 9/50
191/191 - 83s - loss: 0.5093 - accuracy: 0.8231 - val_loss: 0.3944 - val_accuracy: 0.8733
Epoch 10/50
191/191 - 83s - loss: 0.4816 - accuracy: 0.8325 - val_loss: 0.4452 - val_accura

In [31]:
probs = Xception_10.predict(test_dataset299)

In [32]:
probs

array([[1.18627206e-07, 1.20628911e-07, 8.67798924e-04, 3.36065204e-06,
        4.65487719e-05, 9.99081969e-01],
       [8.85337770e-01, 2.03542513e-04, 1.68135238e-03, 1.10408992e-01,
        1.50760100e-03, 8.60724656e-04],
       [1.03369064e-04, 9.98573542e-01, 1.27541542e-03, 5.43339411e-06,
        4.13511079e-05, 8.85504051e-07],
       ...,
       [7.13247017e-09, 9.99988198e-01, 1.17397976e-05, 4.75969628e-08,
        5.53768587e-10, 2.79845203e-10],
       [6.40492067e-02, 5.96018566e-04, 2.68951785e-02, 8.13652694e-01,
        2.13847235e-02, 7.34222680e-02],
       [3.35683092e-03, 5.08859375e-05, 5.62456204e-03, 1.47494718e-01,
        9.36805853e-04, 8.42536211e-01]], dtype=float32)

In [33]:
score = Xception_10.evaluate(test_dataset299)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 10s 146ms/step - loss: 0.3054 - accuracy: 0.8961
Test loss 0.30535322427749634
Test accuracy 0.8961352705955505


##### **Nº de épocas: 50 + 50**

In [34]:
from keras.callbacks import EarlyStopping

Xception_10.fit(
    train_dataset299, # Training data
    validation_data=val_dataset299,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=50, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/50
191/191 - 83s - loss: 0.3075 - accuracy: 0.8944 - val_loss: 0.3478 - val_accuracy: 0.8971
Epoch 2/50
191/191 - 83s - loss: 0.2845 - accuracy: 0.8995 - val_loss: 0.3650 - val_accuracy: 0.8819
Epoch 3/50
191/191 - 83s - loss: 0.3062 - accuracy: 0.8941 - val_loss: 0.3647 - val_accuracy: 0.8805
Epoch 4/50
191/191 - 83s - loss: 0.3036 - accuracy: 0.8897 - val_loss: 0.3400 - val_accuracy: 0.8885
Epoch 5/50
191/191 - 83s - loss: 0.3031 - accuracy: 0.8905 - val_loss: 0.3449 - val_accuracy: 0.8876
Epoch 6/50
191/191 - 83s - loss: 0.3046 - accuracy: 0.8944 - val_loss: 0.3529 - val_accuracy: 0.8824
Epoch 7/50
191/191 - 83s - loss: 0.2925 - accuracy: 0.8989 - val_loss: 0.3346 - val_accuracy: 0.8923
Epoch 8/50
191/191 - 83s - loss: 0.2796 - accuracy: 0.9045 - val_loss: 0.3604 - val_accuracy: 0.8914
Epoch 9/50
191/191 - 83s - loss: 0.2990 - accuracy: 0.8979 - val_loss: 0.3444 - val_accuracy: 0.8899
Epoch 10/50
191/191 - 83s - loss: 0.2883 - accuracy: 0.8995 - val_loss: 0.3468 - val_accura

In [35]:
probs = Xception_10.predict(test_dataset299)

In [36]:
probs

array([[9.74953000e-05, 6.96649181e-07, 1.49692729e-04, 9.98509467e-01,
        1.23137410e-03, 1.12065763e-05],
       [8.33027928e-13, 5.83586431e-16, 6.92941147e-13, 2.34400628e-12,
        1.00000000e+00, 9.70207248e-10],
       [3.04816453e-16, 3.80840365e-18, 2.88884819e-11, 2.02111048e-16,
        1.21254320e-12, 1.00000000e+00],
       ...,
       [5.12775150e-04, 3.09812069e-01, 2.29609553e-02, 1.02298521e-03,
        8.92471639e-04, 6.64798796e-01],
       [3.90470050e-05, 6.34582602e-06, 3.55994445e-03, 2.43122177e-03,
        1.48798063e-04, 9.93814707e-01],
       [1.57969898e-05, 1.30352285e-02, 9.86312568e-01, 5.20277070e-04,
        4.84380234e-06, 1.11280766e-04]], dtype=float32)

In [37]:
score = Xception_10.evaluate(test_dataset299)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 10s 145ms/step - loss: 0.3089 - accuracy: 0.9024
Test loss 0.3089087903499603
Test accuracy 0.9024154543876648


##### **Nº de épocas: 50 + 50 + 50**

In [38]:
from keras.callbacks import EarlyStopping

Xception_10.fit(
    train_dataset299, # Training data
    validation_data=val_dataset299,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=50, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/50
191/191 - 83s - loss: 0.2672 - accuracy: 0.9079 - val_loss: 0.3388 - val_accuracy: 0.8952
Epoch 2/50
191/191 - 83s - loss: 0.2752 - accuracy: 0.9015 - val_loss: 0.3816 - val_accuracy: 0.8857
Epoch 3/50
191/191 - 83s - loss: 0.2694 - accuracy: 0.9092 - val_loss: 0.3961 - val_accuracy: 0.8786
Epoch 4/50
191/191 - 83s - loss: 0.2626 - accuracy: 0.9064 - val_loss: 0.3652 - val_accuracy: 0.8786
Epoch 5/50
191/191 - 83s - loss: 0.2509 - accuracy: 0.9097 - val_loss: 0.3819 - val_accuracy: 0.8748
Epoch 6/50
191/191 - 83s - loss: 0.2675 - accuracy: 0.9094 - val_loss: 0.3650 - val_accuracy: 0.8866
Epoch 7/50
191/191 - 83s - loss: 0.2525 - accuracy: 0.9073 - val_loss: 0.3542 - val_accuracy: 0.8928
Epoch 8/50
191/191 - 83s - loss: 0.2598 - accuracy: 0.9132 - val_loss: 0.3942 - val_accuracy: 0.8805
Epoch 9/50
191/191 - 83s - loss: 0.2562 - accuracy: 0.9122 - val_loss: 0.3706 - val_accuracy: 0.8895
Epoch 10/50
191/191 - 83s - loss: 0.2566 - accuracy: 0.9137 - val_loss: 0.3770 - val_accura

In [39]:
probs = Xception_10.predict(test_dataset299)

In [40]:
probs

array([[9.9929142e-01, 6.8384054e-08, 8.3365139e-06, 6.8162096e-04,
        1.3849587e-05, 4.6370978e-06],
       [1.1549933e-02, 5.8126204e-07, 3.3914373e-04, 3.1851768e-04,
        3.6775210e-04, 9.8742408e-01],
       [3.5137851e-02, 2.6176520e-02, 4.8096859e-01, 4.7047056e-02,
        3.2363400e-02, 3.7830663e-01],
       ...,
       [9.0010273e-01, 3.6528561e-06, 1.0921769e-04, 9.8520711e-02,
        8.6261437e-04, 4.0102482e-04],
       [2.8510609e-05, 2.5033564e-06, 9.9994135e-01, 2.0801377e-05,
        5.4946699e-06, 1.3392091e-06],
       [2.3601269e-03, 4.7729854e-03, 2.6802598e-02, 3.9099134e-03,
        9.6150720e-01, 6.4710109e-04]], dtype=float32)

In [41]:
score = Xception_10.evaluate(test_dataset299)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 10s 148ms/step - loss: 0.2983 - accuracy: 0.9063
Test loss 0.298338383436203
Test accuracy 0.9062802195549011


##### **Nº de épocas: 50 + 50 + 50 + 50**

In [42]:
from keras.callbacks import EarlyStopping

Xception_10.fit(
    train_dataset299, # Training data
    validation_data=val_dataset299,
    batch_size=32, # Batch size for the optimizer algorithm
    epochs=50, # Number of epochs to run the optimizer algorithm
    callbacks=[
        EarlyStopping(patience=20, restore_best_weights=True)               
    ],
    verbose=2 # Level of verbosity of the log messages
)

Epoch 1/50
191/191 - 83s - loss: 0.2581 - accuracy: 0.9087 - val_loss: 0.3566 - val_accuracy: 0.8899
Epoch 2/50
191/191 - 83s - loss: 0.2663 - accuracy: 0.9066 - val_loss: 0.3473 - val_accuracy: 0.8914
Epoch 3/50
191/191 - 83s - loss: 0.2708 - accuracy: 0.9061 - val_loss: 0.3807 - val_accuracy: 0.8833
Epoch 4/50
191/191 - 83s - loss: 0.2854 - accuracy: 0.9032 - val_loss: 0.3901 - val_accuracy: 0.8838
Epoch 5/50
191/191 - 83s - loss: 0.2668 - accuracy: 0.9066 - val_loss: 0.3909 - val_accuracy: 0.8843
Epoch 6/50
191/191 - 83s - loss: 0.2703 - accuracy: 0.9084 - val_loss: 0.3390 - val_accuracy: 0.8904
Epoch 7/50
191/191 - 83s - loss: 0.2533 - accuracy: 0.9114 - val_loss: 0.3814 - val_accuracy: 0.8857
Epoch 8/50
191/191 - 83s - loss: 0.2653 - accuracy: 0.9081 - val_loss: 0.3597 - val_accuracy: 0.8871
Epoch 9/50
191/191 - 83s - loss: 0.2586 - accuracy: 0.9112 - val_loss: 0.3458 - val_accuracy: 0.8928
Epoch 10/50
191/191 - 83s - loss: 0.2426 - accuracy: 0.9150 - val_loss: 0.3502 - val_accura

In [43]:
probs = Xception_10.predict(test_dataset299)

In [44]:
probs

array([[1.9607421e-02, 5.1900349e-04, 9.9962968e-03, 9.6207148e-01,
        4.6953617e-04, 7.3362472e-03],
       [1.7687078e-05, 1.8729182e-06, 9.5011288e-04, 2.9217533e-04,
        9.4929838e-01, 4.9439888e-02],
       [7.5429962e-03, 7.4959683e-01, 2.3058675e-01, 8.6206188e-03,
        2.4295936e-03, 1.2232118e-03],
       ...,
       [4.1726927e-31, 2.6253481e-38, 4.6247807e-38, 8.5097145e-38,
        1.0000000e+00, 6.4483641e-38],
       [2.1838639e-07, 1.6965743e-04, 9.9981123e-01, 2.8602522e-06,
        1.2464875e-07, 1.5965108e-05],
       [2.8991927e-03, 1.4049674e-02, 8.8600737e-01, 9.5658094e-02,
        9.2295336e-04, 4.6271365e-04]], dtype=float32)

In [45]:
score = Xception_10.evaluate(test_dataset299)
print("Test loss", score[0])
print("Test accuracy", score[1])

65/65 [==============================] - 10s 145ms/step - loss: 0.2955 - accuracy: 0.9092
Test loss 0.2955189049243927
Test accuracy 0.9091787338256836


#### **Conclusiones:**

Dado que el modelo **"Xception_10" entrenado con 100 épocas (50 + 50)** alcanza el rendimiento buscado se toma como válido.

Se ha continuado entrenando el modelo en bloques de 50 épocas para comprobar si se conseguía mejorar el rendimiento. Éste aumenta, pero la mejora podría considerarse poco significativa.

## Summary of results

Example results table

|Image processing|Neural network model|Training strategy|Test accuracy|
|----------------|--------------------|-----------------|-------------|
|Size 32x32, batch size 16|Convolutional(32) + Flatten + Dense(64)|Train from scratch|xx%|
|Size 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|

A continuación, se muestran las arquitecturas de las redes definidas.

### **"LeNet"**



##### **"LeNet_01" (RED NEURONAL CONVOLUCIONAL)**



In [156]:
LeNet_01.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization_16 (Batc (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)         

##### **"LeNet_02" (RED NEURONAL CONVOLUCIONAL)**



In [221]:
LeNet_02.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_21 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_zoom_7 (RandomZoom)   (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_21 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_translation_7 (Random (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_8 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 126, 126, 64)      1792      
_________________________________________________________________
batch_normalization_87 (Batc (None, 126, 126, 64)    

##### **"LeNet_03" (RED NEURONAL CONVOLUCIONAL)**



In [222]:
LeNet_03.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_22 (RandomFlip)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom_8 (RandomZoom)   (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation_22 (RandomRo (None, 256, 256, 3)       0         
_________________________________________________________________
random_translation_8 (Random (None, 256, 256, 3)       0         
_________________________________________________________________
random_crop_5 (RandomCrop)   (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_9 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 126, 126, 64)    

##### **"LeNet_04" (RED NEURONAL CONVOLUCIONAL)**



In [223]:
LeNet_04.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_28 (RandomFlip)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom_13 (RandomZoom)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation_28 (RandomRo (None, 256, 256, 3)       0         
_________________________________________________________________
random_translation_13 (Rando (None, 256, 256, 3)       0         
_________________________________________________________________
random_crop_10 (RandomCrop)  (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_14 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 125, 125, 120)   

##### **"LeNet_05" (RED NEURONAL CONVOLUCIONAL)**



In [224]:
LeNet_05.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_24 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_zoom_10 (RandomZoom)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_24 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_translation_10 (Rando (None, 128, 128, 3)       0         
_________________________________________________________________
random_crop_7 (RandomCrop)   (None, 128, 128, 3)       0         
_________________________________________________________________
rescaling_11 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 125, 125, 128)   

##### **"LeNet_06" (RED NEURONAL CONVOLUCIONAL)**



In [225]:
LeNet_06.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_25 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_zoom_11 (RandomZoom)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_25 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_translation_11 (Rando (None, 128, 128, 3)       0         
_________________________________________________________________
random_crop_8 (RandomCrop)   (None, 85, 85, 3)         0         
_________________________________________________________________
rescaling_12 (Rescaling)     (None, 85, 85, 3)         0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 83, 83, 32)      

##### **"LeNet_07" (RED NEURONAL CONVOLUCIONAL)**



In [258]:
LeNet_07.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_34 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_34 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_135 (Bat (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_136 (Bat (None, 61, 61, 64)      

### **"VGG16"**



##### **"VGG16_01" (RED NEURONAL CONVOLUCIONAL)**



In [150]:
vgg16_01.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_10 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_10 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_4 (RandomCon (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 512)               0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)             

##### **"VGG16_02 (RED NEURONAL CONVOLUCIONAL)**



In [149]:
vgg16_02.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_11 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_11 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_5 (RandomCon (None, 128, 128, 3)       0         
_________________________________________________________________
lambda_5 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
batch_normalization_41 (Batc (None, 4, 4, 512)         2048      
_________________________________________________________________
global_average_pooling2d_5 ( (None, 512)             

### **"Xception"**



##### **"Xception_01" (RED NEURONAL CONVOLUCIONAL)**



In [139]:
Xception_01.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 526)               1077774   
_________________________________________________________________
dropout_19 (Dropout)         (None, 526)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 258)             

##### **"Xception_02" (RED NEURONAL CONVOLUCIONAL)**



In [227]:
Xception_02.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_27 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_27 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_14 (RandomCo (None, 128, 128, 3)       0         
_________________________________________________________________
lambda_15 (Lambda)           (None, 128, 128, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_15  (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)            

##### **"Xception_03" (RED NEURONAL CONVOLUCIONAL)**



In [141]:
Xception_03.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_8 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_13 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_13 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_7 (RandomCon (None, 128, 128, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_8 ( (None, 2048)              0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 2048)            

##### **"Xception_04" (RED NEURONAL CONVOLUCIONAL)**



In [142]:
Xception_04.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_9 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_14 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_14 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_8 (RandomCon (None, 128, 128, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_9 ( (None, 2048)              0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 2048)            

##### **"Xception_05" (RED NEURONAL CONVOLUCIONAL)**



In [143]:
Xception_05.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_10 (Lambda)           (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_15 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_15 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_9 (RandomCon (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_62 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_10  (None, 2048)            

##### **"Xception_06" (RED NEURONAL CONVOLUCIONAL)**


In [144]:
Xception_06.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_11 (Lambda)           (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_16 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_16 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_10 (RandomCo (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_67 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_11  (None, 2048)            

##### **"Xception_07" (RED NEURONAL CONVOLUCIONAL)**



In [145]:
Xception_07.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_12 (Lambda)           (None, 128, 128, 3)       0         
_________________________________________________________________
random_flip_17 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
random_rotation_17 (RandomRo (None, 128, 128, 3)       0         
_________________________________________________________________
random_contrast_11 (RandomCo (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_72 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_12  (None, 2048)            

##### **"Xception_08" (RED NEURONAL CONVOLUCIONAL)**



In [146]:
Xception_08.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_13 (Lambda)           (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip_18 (RandomFlip)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation_18 (RandomRo (None, 256, 256, 3)       0         
_________________________________________________________________
random_contrast_12 (RandomCo (None, 256, 256, 3)       0         
_________________________________________________________________
resizing (Resizing)          (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_77 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)      

##### **"Xception_09" (RED NEURONAL CONVOLUCIONAL)**



In [147]:
Xception_09.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_14 (Lambda)           (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip_19 (RandomFlip)  (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation_19 (RandomRo (None, 256, 256, 3)       0         
_________________________________________________________________
random_contrast_13 (RandomCo (None, 256, 256, 3)       0         
_________________________________________________________________
resizing_1 (Resizing)        (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_82 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
xception (Functional)        (None, 4, 4, 2048)      

##### **"Xception_10" (RED NEURONAL CONVOLUCIONAL)**



In [148]:
Xception_10.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 299, 299, 3)       0         
_________________________________________________________________
random_flip_2 (RandomFlip)   (None, 299, 299, 3)       0         
_________________________________________________________________
random_rotation_2 (RandomRot (None, 299, 299, 3)       0         
_________________________________________________________________
random_contrast_2 (RandomCon (None, 299, 299, 3)       0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 299, 299, 3)       12        
_________________________________________________________________
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)             